In [2]:
from selenium import webdriver
from selenium_stealth import stealth
from selenium.webdriver.chrome.service import Service
from selectolax.lexbor import LexborHTMLParser
from typing import List, Dict, Callable
import time, random, re
import requests
import io
from io import BytesIO
from io import StringIO
from bs4 import BeautifulSoup
import glob as glob
from fuzzywuzzy import fuzz
import numpy as np
from pymed import PubMed
import os, json
from typing import List, Dict, Callable
import time, random, re
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time 
import glob
import os
import pandas as pd
import numpy as np

/opt/anaconda3/envs/trimmer_lab/lib/python3.8/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [28]:
!brew install chromedriver
!brew upgrade chromedriver

/bin/bash: brew: command not found


/bin/bash: brew: command not found


# TODO
- clean up functions and doc strings and parameter typing - especially where it says for you to start onwards. 
- clean up the resulting files and merge pubmed results (clean duplicates etc.) 
- we also want to be sure the query is in the paper based on the pubmed url -
  - i think there were some false positives from google scholar and most journals easy to hit url grab pdf and search it
- when grouping by duplicates mark that multiple neuromabs were marked and str join on groupby so we see all the mabs that showed up in that paper. 
- query the paper via pubmed in order to confirm the query is in the paper? (we are doing this to confirm something which i cant rembmer its been a while)
- follow up when you get the stage above and we will be adding more potential query data to the pipeline from antibodies inc and addgene
- make a directory for the dates that were queried and the querieis that were created as files in the directory (unless you have a better idea but we want to be able to abstractly see the queries done and maybe check for files already being there and skip those queries?)   
  - for now this is all the data we need need but lets make somethign for this a bit better for the future. 



## LONG TERM
- data story telling and figures
- add the add gene counts and do something creative to help us understand the most important research reagents (data science exercise)
- antibodies inc sales as well but this and add gene are bottom of the list
- add this data to the database/website

# A few notes
- The code is not very clean but it works. Not sure how much you need of this but would be good excercise to work on and
- `brew install chromedriver` to install chromedriver
- change the directory above for your outputs 
- I can give you my login for VPN but you can also run with sleep as a background task - i think it does take a couple hours to reset. 
- Follow ups is find the papers with pubmed links and then scrape the pubmed links for the abstracts.

# Begin the scrapping of google scholar citations for neuromabs

In [3]:
query_out = '/Users/keithmitchell/Desktop/Repositories/NeuroMabSeq/trimmer/picklequeries/new_test3/'

In [4]:
def parse(parser: Callable, organic_results_data: Callable):
    '''
    Arugments:
    - parser:  Lexbor parser from scrape_google_scholar_organic_results() function.
    - organic_results_data: List to append data to. List origin location is scrape_google_scholar_organic_results() function. Line 104.
    
    This function parses data from Google Scholar Organic results and appends data to a List.
    
    It's used by scrape_google_scholar_organic_results().
    
    It returns nothing as it appends data to `organic_results_data`, 
    which appends it to `organic_results_data` List in the scrape_google_scholar_organic_results() function.
    '''
    
    for result in parser.css('.gs_r.gs_or.gs_scl'):
        try:
            title: str = result.css_first('.gs_rt').text()
        except: title = None

        try:
            title_link: str = result.css_first('.gs_rt a').attrs['href']
        except: title_link = None

        try:
            publication_info: str = result.css_first('.gs_a').text()
        except: publication_info = None

        try:
            snippet: str = result.css_first('.gs_rs').text()
        except: snippet = None

        try:
            # if Cited by is present in inline links, it will be extracted
            cited_by_link = ''.join([link.attrs['href'] for link in result.css('.gs_ri .gs_fl a') if 'Cited by' in link.text()])
        except: cited_by_link = None
        
        try:
            # if Cited by is present in inline links, it will be extracted and type cast it to integer
            cited_by_count = int(''.join([re.search(r'\d+', link.text()).group() for link in result.css('.gs_ri .gs_fl a') if 'Cited by' in link.text()]))
        except: cited_by_count = None
        
        try:
            pdf_file: str = result.css_first('.gs_or_ggsm a').attrs['href']
        except: pdf_file = None

        organic_results_data.append({
            'title': title,
            'title_link': title_link,
            'publication_info': publication_info,
            'snippet': snippet if snippet else None,
            'cited_by_link': f'https://scholar.google.com{cited_by_link}' if cited_by_link else None,
            'cited_by_count': cited_by_count if cited_by_count else None,
            'pdf_file': pdf_file
        })

def scrape_google_scholar_organic_results(
                                        query: str,
                                        pagination: bool = False,
                                        operating_system: str = 'Windows' or 'Linux',
                                        year_start: int = None,
                                        year_end: int = None,
                                        save_to_csv: bool = False, 
                                        save_to_json: bool = False
                                        ) -> List[Dict[str, str]]:
    '''
    Extracts data from Google Scholar Organic resutls page:
    - title: str
    - title_link: str
    - publication_info: str 
    - snippet: str
    - cited_by_link: str 
    - cited_by_count: int
    - pdf_file: str
    
    Arguments:
    - query: str. Search query. 
    - pagination: bool. Enables or disables pagination.
    - operating_system: str. 'Windows' or 'Linux', Checks for operating system to either run Windows or Linux verson of chromedriver
    
    Usage:
    data = scrape_google_scholar_organic_results(query='blizzard', pagination=False, operating_system='win') # pagination defaults to False 
    
    for organic_result in data:
        print(organic_result['title'])
        print(organic_result['pdf_file'])
    '''
    assert year_start and year_end
    if year_start or year_end:
        assert year_start and year_end
        assert year_start <= year_end

    from selenium import webdriver
    from selenium.webdriver.chrome.service import Service

    service = Service()
    #service = Service(executable_path='/usr/local/bin/chromedriver')


    # selenium stealth
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    
    options.add_experimental_option('excludeSwitches', ['enable-automation'])
    options.add_experimental_option('useAutomationExtension', False)
    
    # checks for operating system to either run Windows or Linux verson of chromedriver
    # expects to have chromedriver near the runnable file
    if operating_system is None:
        raise Exception('Please provide your OS to `operating_system` argument: "Windows" or "Linux" for script to operate.')
    
    # if operating_system.lower() == 'windows' or 'win':
    #     driver = webdriver.Chrome(options=options, service=Service(executable_path='chromedriver.exe'))
    
    #if operating_system.lower() == 'linux': 
    #driver = webdriver.Chrome(options=options, service=Service(executable_path='/Applications/Google Chrome.app/Contents/MacOS/Google Chrome'))
    


    driver = webdriver.Chrome(service=service, options=options)
    stealth(driver,
        languages=['en-US', 'en'],
        vendor='Google Inc.',
        platform='Win32',
        webgl_vendor='Intel Inc.',
        renderer='Intel Iris OpenGL Engine',
        fix_hairline=True,
    )
    
    page_num: int = 0
    organic_results_data: list = []
    
    if pagination:
        while True:
            # parse all pages
            driver.get(f'https://scholar.google.com/scholar?q={query}&hl=en&gl=us&start={page_num}&as_ylo={year_start}&as_yhi={year_end}')
            parser = LexborHTMLParser(driver.page_source)
            
            parse(parser=parser, organic_results_data=organic_results_data)
            
            # pagination
            if parser.css_first('.gs_ico_nav_next'):  # checks for the "Next" page button
                page_num += 10                        # paginate to the next page
                time.sleep(random.randint(1, 2))      # sleep between paginations
            else:
                break
    else:
        # parse single, first page
        driver.get(f'https://scholar.google.com/scholar?q={query}&hl=en&gl=us&start={page_num}')
        parser = LexborHTMLParser(driver.page_source)
    
        parse(parser=parser, organic_results_data=organic_results_data)
        
    if save_to_csv:
        pd.DataFrame(data=organic_results_data).to_csv('google_scholar_organic_results_data.csv', 
                                                        index=False, encoding='utf-8')
    if save_to_json:
        pd.DataFrame(data=organic_results_data).to_json('google_scholar_organic_results_data.json', 
                                                        index=False, orient='records')
    driver.quit()
    return organic_results_data


def AddTargetCol(df,final_df):
            
    merged_df = final_df.merge(df[['Clone', 'Target']], left_on=['Antibody'],right_on=['Clone'], how='left')
    merged_df = merged_df.drop('Clone', axis=1)
   
    return merged_df

def CreatURL(query):
    
    query1 = query[0:8]
    end_pos = query.find('/', 9)
    query2= query[9:end_pos]
    query3= query[end_pos+1:]
    
    url = 'https://scholar.google.com/scholar?hl=en&as_sdt=0%2C5&q=' + (query1 + '+' + query2 + '%' + '2F' + query3 + '&oq=neuromab+')

    return url

def CreateDF(query,result_value,data):
    
    # Record the difference between search result # and data length 
    # TODO wendon not sure why pop but:
        # i bet this whole function can be cleaned quite a bit and might be a good initial cleaning stage
        # more info is still better than less info most of the time moving forward more filtering at the final result after pubmed query
    count = (int(result_value)) - (int({len(data)}.pop()))
    final_df = pd.DataFrame(data)
    
    # Get rid of extraneous strings from title 
    final_df['title'] = final_df['title'].str.replace('\[HTML\]\[HTML\]|\[PDF\]\[PDF\]|\[CITATION\]|\[C\]|\[BOOK\]\[BOOK\]|\[B\]|\[BOOK\]\[B\]', '')

    # Creat file for data 
    file_query = query.replace("/","_")
    file_query = file_query[0:8]+file_query[9:]
    filename = file_query + ".csv"
    directory = query_out

    # Input Antibody name into table 
    final_df["Antibody"] = [query[9:]] * len(final_df)
    final_df = AddTargetCol(df,final_df)
    final_filename = directory + filename
    final_df.to_csv(final_filename, index=False)


def run_code(new_index, df):
    # Initialize a value to count 0 result pages 
    zero_result_counter = 0
    count_list = []
    # in the future we will only need to run 2023 - 202X
    zero_result_length = 10
    start_year = 2023
    end_year = 2025


    for index, query_name in zip(df.index[new_index:], df['Clone'][new_index:]):
        # Construct the filename and directory path
        file_query = 'neuromab' + ' ' + query_name.replace("/", "_")
        filename = file_query[0:8] + file_query[9:] + ".csv"
        directory = query_out
        final_filename = directory + filename

        # Check if the file already exists
        print(final_filename)
        if os.path.exists(final_filename):
            print(f"File {final_filename} already exists. Skipping query for {query_name}.")
            count_list = []
            continue  # Skip this iteration and move to the next query_name

        data = {}
        print(index, query_name)
        # Set query for google scholar 
        query = 'neuromab' + ' ' + query_name


        # Grab data from google scholar 
        data = scrape_google_scholar_organic_results(query=query, pagination=True, operating_system="Windows", year_start=start_year, year_end=end_year)
        if len(data) == 0:
            count_list.append(len(data))
            past_0 = 30
            # if the last X elements are 0 then break - we got booted and need to reset the vpn
            # check if the last past_0 elements are 0
            if len(count_list) > past_0 and all([i == 0 for i in count_list[-past_0:]]):
                print("We got booted, resetting vpn")
                new_index = index - past_0
                print(count_list[-20:])
                return new_index
            continue
        else:
            print(len(data))
            count_list.append(len(data))
        # create URL to grab results 
        url = CreatURL(query)
        url_info = requests.get(url)
        html = url_info.text

        # Create a BeautifulSoup object from the HTML string
        soup = BeautifulSoup(html, 'html.parser')
        result_elem = soup.select_one('#gs_ab_md .gs_ab_mdw')

        if result_elem:
            result_text = result_elem.get_text(strip=True)
            result_match = re.search(r'(\d+[\d,]*)\s+results', result_text)
            if result_match == None:
                result_match = re.search(r'(\d+[\d,]*)\s+result', result_text)

            if result_match:
                result_value = result_match.group(1)
                final_df = CreateDF(query,result_value,data)
            elif len(data) > 0:
                print(query + ' ' + "Problem with search")
            else:
                print(query + ' ' +"Search - No papers Available")

In [24]:
new_index = run_code(0, df)
print(new_index)

/Users/keithmitchell/Desktop/Repositories/NeuroMabSeq/trimmer/picklequeries/new_test3/neuromabN358_68.csv
0 N358/68
/Users/keithmitchell/Desktop/Repositories/NeuroMabSeq/trimmer/picklequeries/new_test3/neuromabN297_59.csv
1 N297/59
1


<ipython-input-16-2332065dedf2>:193: FutureWarning: The default value of regex will change from True to False in a future version.
  final_df['title'] = final_df['title'].str.replace('\[HTML\]\[HTML\]|\[PDF\]\[PDF\]|\[CITATION\]|\[C\]|\[BOOK\]\[BOOK\]|\[B\]|\[BOOK\]\[B\]', '')


/Users/keithmitchell/Desktop/Repositories/NeuroMabSeq/trimmer/picklequeries/new_test3/neuromabN144_14.csv
2 N144/14
18


<ipython-input-16-2332065dedf2>:193: FutureWarning: The default value of regex will change from True to False in a future version.
  final_df['title'] = final_df['title'].str.replace('\[HTML\]\[HTML\]|\[PDF\]\[PDF\]|\[CITATION\]|\[C\]|\[BOOK\]\[BOOK\]|\[B\]|\[BOOK\]\[B\]', '')


/Users/keithmitchell/Desktop/Repositories/NeuroMabSeq/trimmer/picklequeries/new_test3/neuromabN441_35.csv
3 N441/35
/Users/keithmitchell/Desktop/Repositories/NeuroMabSeq/trimmer/picklequeries/new_test3/neuromabN46_30.csv
4 N46/30
9


...

/Users/keithmitchell/Desktop/Repositories/NeuroMabSeq/trimmer/picklequeries/new_test3/neuromabN411_51.csv
28 N411/51
1


<ipython-input-16-2332065dedf2>:193: FutureWarning: The default value of regex will change from True to False in a future version.
  final_df['title'] = final_df['title'].str.replace('\[HTML\]\[HTML\]|\[PDF\]\[PDF\]|\[CITATION\]|\[C\]|\[BOOK\]\[BOOK\]|\[B\]|\[BOOK\]\[B\]', '')


/Users/keithmitchell/Desktop/Repositories/NeuroMabSeq/trimmer/picklequeries/new_test3/neuromabN271_44.csv
29 N271/44
9


<ipython-input-16-2332065dedf2>:193: FutureWarning: The default value of regex will change from True to False in a future version.
  final_df['title'] = final_df['title'].str.replace('\[HTML\]\[HTML\]|\[PDF\]\[PDF\]|\[CITATION\]|\[C\]|\[BOOK\]\[BOOK\]|\[B\]|\[BOOK\]\[B\]', '')


/Users/keithmitchell/Desktop/Repositories/NeuroMabSeq/trimmer/picklequeries/new_test3/neuromabN347_42.csv
30 N347/42
/Users/keithmitchell/Desktop/Repositories/NeuroMabSeq/trimmer/picklequeries/new_test3/neuromabN76_8.csv
31 N76/8
/Users/keithmitchell/Desktop/Repositories/NeuroMabSeq/trimmer/picklequeries/new_test3/neuromabN76_3.csv
32 N76/3
/Users/keithmitchell/Desktop/Repositories/NeuroMabSeq/trimmer/picklequeries/new_test3/neuromabN65_37.csv
33 N65/37
/Users/keithmitchell/Desktop/Repositories/NeuroMabSeq/trimmer/picklequeries/new_test3/neuromabN360A_24.csv
34 N360A/24
/Users/keithmitchell/Desktop/Repositories/NeuroMabSeq/trimmer/picklequeries/new_test3/neuromabN336B_83.csv
35 N336B/83
/Users/keithmitchell/Desktop/Repositories/NeuroMabSeq/trimmer/picklequeries/new_test3/neuromabN332B_15.csv
36 N332B/15
/Users/keithmitchell/Desktop/Repositories/NeuroMabSeq/trimmer/picklequeries/new_test3/neuromabL124_59.csv
37 L124/59
/Users/keithmitchell/Desktop/Repositories/NeuroMabSeq/trimmer/pickle

In [25]:
len(df)

483

In [43]:
new_index = run_code(461, df)
print(new_index)

/Users/keithmitchell/Desktop/Repositories/NeuroMabSeq/trimmer/picklequeries/new_test3/neuromabN479_107.csv
File /Users/keithmitchell/Desktop/Repositories/NeuroMabSeq/trimmer/picklequeries/new_test3/neuromabN479_107.csv already exists. Skipping query for N479/107.
/Users/keithmitchell/Desktop/Repositories/NeuroMabSeq/trimmer/picklequeries/new_test3/neuromabN152B_23.csv
462 N152B/23
/Users/keithmitchell/Desktop/Repositories/NeuroMabSeq/trimmer/picklequeries/new_test3/neuromabN351_23.csv
463 N351/23
/Users/keithmitchell/Desktop/Repositories/NeuroMabSeq/trimmer/picklequeries/new_test3/neuromabN351_24.csv
464 N351/24
/Users/keithmitchell/Desktop/Repositories/NeuroMabSeq/trimmer/picklequeries/new_test3/neuromabL118_80.csv
465 L118/80
1


<ipython-input-33-c56e2efc4f66>:193: FutureWarning: The default value of regex will change from True to False in a future version.
  final_df['title'] = final_df['title'].str.replace('\[HTML\]\[HTML\]|\[PDF\]\[PDF\]|\[CITATION\]|\[C\]|\[BOOK\]\[BOOK\]|\[B\]|\[BOOK\]\[B\]', '')


/Users/keithmitchell/Desktop/Repositories/NeuroMabSeq/trimmer/picklequeries/new_test3/neuromabL118_66.csv
466 L118/66
/Users/keithmitchell/Desktop/Repositories/NeuroMabSeq/trimmer/picklequeries/new_test3/neuromabN28_9.csv
467 N28/9
9


<ipython-input-33-c56e2efc4f66>:193: FutureWarning: The default value of regex will change from True to False in a future version.
  final_df['title'] = final_df['title'].str.replace('\[HTML\]\[HTML\]|\[PDF\]\[PDF\]|\[CITATION\]|\[C\]|\[BOOK\]\[BOOK\]|\[B\]|\[BOOK\]\[B\]', '')


/Users/keithmitchell/Desktop/Repositories/NeuroMabSeq/trimmer/picklequeries/new_test3/neuromabN29_29.csv
468 N29/29
/Users/keithmitchell/Desktop/Repositories/NeuroMabSeq/trimmer/picklequeries/new_test3/neuromabN34_34.csv
469 N34/34
/Users/keithmitchell/Desktop/Repositories/NeuroMabSeq/trimmer/picklequeries/new_test3/neuromabL108_92.csv
470 L108/92
1


<ipython-input-33-c56e2efc4f66>:193: FutureWarning: The default value of regex will change from True to False in a future version.
  final_df['title'] = final_df['title'].str.replace('\[HTML\]\[HTML\]|\[PDF\]\[PDF\]|\[CITATION\]|\[C\]|\[BOOK\]\[BOOK\]|\[B\]|\[BOOK\]\[B\]', '')


/Users/keithmitchell/Desktop/Repositories/NeuroMabSeq/trimmer/picklequeries/new_test3/neuromabN440_21.csv
471 N440/21
/Users/keithmitchell/Desktop/Repositories/NeuroMabSeq/trimmer/picklequeries/new_test3/neuromabN440_61.csv
472 N440/61
/Users/keithmitchell/Desktop/Repositories/NeuroMabSeq/trimmer/picklequeries/new_test3/neuromabN449_73.csv
473 N449/73
/Users/keithmitchell/Desktop/Repositories/NeuroMabSeq/trimmer/picklequeries/new_test3/neuromabN432_21.csv
474 N432/21
1


<ipython-input-33-c56e2efc4f66>:193: FutureWarning: The default value of regex will change from True to False in a future version.
  final_df['title'] = final_df['title'].str.replace('\[HTML\]\[HTML\]|\[PDF\]\[PDF\]|\[CITATION\]|\[C\]|\[BOOK\]\[BOOK\]|\[B\]|\[BOOK\]\[B\]', '')


/Users/keithmitchell/Desktop/Repositories/NeuroMabSeq/trimmer/picklequeries/new_test3/neuromabN432_63.csv
475 N432/63
/Users/keithmitchell/Desktop/Repositories/NeuroMabSeq/trimmer/picklequeries/new_test3/neuromabK91_36.csv
476 K91/36
/Users/keithmitchell/Desktop/Repositories/NeuroMabSeq/trimmer/picklequeries/new_test3/neuromabN391_68.csv
477 N391/68
/Users/keithmitchell/Desktop/Repositories/NeuroMabSeq/trimmer/picklequeries/new_test3/neuromabN428_12.csv
478 N428/12
/Users/keithmitchell/Desktop/Repositories/NeuroMabSeq/trimmer/picklequeries/new_test3/neuromabN286_74.csv
479 N286/74
/Users/keithmitchell/Desktop/Repositories/NeuroMabSeq/trimmer/picklequeries/new_test3/neuromabK88_64.csv
480 K88/64
/Users/keithmitchell/Desktop/Repositories/NeuroMabSeq/trimmer/picklequeries/new_test3/neuromabN476_9.csv
481 N476/9
/Users/keithmitchell/Desktop/Repositories/NeuroMabSeq/trimmer/picklequeries/new_test3/neuromabN328B_37.csv
482 N328B/37
None


# Add target column from neuromab database


In [5]:
data = requests.get("https://neuromab.ucdavis.edu/catalog-download.cfm").content
df = pd.read_csv(BytesIO(data))
df

,TargetType,Target,AccessionNum,Clone,HumanGeneName,DataSheetFileName,IsoType,ValidationT,ValidationBrIB,ValidationBrIHC,ValidationKO,RRIDAB
0,Epigenetics,5-formylcytidine,NaN,N358/68,----,N358_68.pdf,IgG1,NaN,NaN,NaN,NaN,2877251
1,Epigenetics,5-hydroxymethylcytidine,NaN,N297/59,----,N297_59.pdf,IgG2a,NaN,NaN,NaN,NaN,2877253
2,Tag,6xHis,NaN,N144/14,----,N144_14.pdf,IgG1,Pass,NaN,NaN,NaN,2877254
3,Receptors,ADAM11,Q9R1V4,N441/35,ADAM11,N441_35.pdf,IgG2b,Pass,Pass,Pass,ND,2877610
4,Receptors,ADAM22 (cytoplasmic),Q9R1V6,N46/30,ADAM22,N46_30.pdf,IgG2b,Pass,Pass,Pass,Pass,2877292
...,...,...,...,...,...,...,...,...,...,...,...,...
478,Rare Disease,WRN,Q14191,N428/12,WRN,N428_12.pdf,IgG1,Pass,Pass,Pass,ND,2877600
479,Other,Zebrafish PSD Marker,NaN,N286/74,----,N286_74.pdf,IgG1,ND,ND,ND,ND,2877461
480,Other,Zinedin,Q9NRL3,K88/64,STRN4,K88_64.pdf,IgG2a,Pass,Pass,Pass,ND,2877335
481,Transporters,ZIP3,Q99K24,N476/9,SLC39A3,N476_9.pdf,IgG1,Pass,Pass,Pass,ND,2716255


# Merge all previous runs and the current run into one dataframe

In [6]:
# get data file names - replace with your path for what i send
import glob
import pandas as pd

path = r'/Users/keithmitchell/Desktop/Repositories/NeuroMabSeq/trimmer/picklequeries/new_test'
filenames = glob.glob(path + "/*.csv")

dfs = []
for filename in filenames:
    dfs.append(pd.read_csv(filename))
allnmab_df = pd.concat(dfs, ignore_index=True)

In [7]:
allnmab_df

,title,title_link,publication_info,snippet,cited_by_link,cited_by_count,pdf_file,Antibody,Target
0,Deubiquitylating enzyme USP2 counteracts Nedd4...,https://www.sciencedirect.com/science/article/...,"K Krzystanek, HB Rasmussen, M Grunnet, O Staub...","BACKGROUND: KCNQ1 (Kv7.1), together with its K...",https://scholar.google.com/scholar?cites=98014...,35.0,NaN,N37A/10,Kv7.1/KCNQ1 K+ channel
1,Caracterización de los canales de potasio depe...,https://idus.us.es/handle/11441/138533,M Ortego Domínguez - 2022 - idus.us.es,Los adipocitos blancos son células que partici...,NaN,NaN,https://idus.us.es/bitstream/handle/11441/1385...,N375/67,Kv3.3 K+ channel
2,Neuroinflammation alters integrative propertie...,https://link.springer.com/article/10.1007/s120...,"F Frigerio, C Flynn, Y Han, K Lyman, JN Lugo… ...","… , N212/17, Neuromab, CA, USA) antibodies usi...",https://scholar.google.com/scholar?cites=19097...,41.0,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6...,N212/17,TRIP8b (constant)
3,Allostery between two binding sites in the io...,https://www.jbc.org/article/S0021-9258(20)3303...,"KA Lyman, Y Han, RJ Heuermann, X Cheng… - Jour...",Tetratricopeptide repeat (TPR) domains are ubi...,https://scholar.google.com/scholar?cites=14199...,12.0,https://www.jbc.org/article/S0021-9258(20)3303...,N212/17,TRIP8b (constant)
4,Distal dendritic enrichment of HCN1 channels i...,https://www.eneuro.org/content/5/5/eneuro.0258...,"M Meseke, F Neumüller, B Brunne, X Li, M Anstö...",HCN1 compartmentalization in CA1 pyramidal cel...,https://scholar.google.com/scholar?cites=50206...,8.0,https://www.eneuro.org/content/eneuro/5/5/ENEU...,N212/17,TRIP8b (constant)
...,...,...,...,...,...,...,...,...,...
3783,Distinct but overlapping roles of LRRTM1 and L...,https://elifesciences.org/articles/64742,"SH Dhume, SA Connor, F Mills, PK Tari… - Elife...",LRRTMs are postsynaptic cell adhesion proteins...,https://scholar.google.com/scholar?cites=97818...,5.0,https://elifesciences.org/articles/64742.pdf,N127/31,Pan-SAPAP
3784,Synaptic protein interaction networks encode ...,https://www.cell.com/cell-reports/pdf/S2211-12...,"JD Lautz, KB Tsegay, Z Zhu, EP Gniffke, JP Wel...",A core network of widely expressed proteins wi...,https://scholar.google.com/scholar?cites=12974...,6.0,https://www.cell.com/cell-reports/pdf/S2211-12...,N127/31,Pan-SAPAP
3785,Pannexin 2 protein expression is not restrict...,https://www.frontiersin.org/articles/10.3389/f...,"M Le Vasseur, J Lelowski, JF Bechberger… - Fro...","… by UC Davis/NIH NeuroMab Facility (Davis, CA...",https://scholar.google.com/scholar?cites=77804...,81.0,https://www.frontiersin.org/articles/10.3389/f...,N121A/1,Pannexin-2
3786,The role of Pannexin 2 in mitochondrial functi...,https://open.library.ubc.ca/soa/cIRcle/collect...,M Le Vasseur - 2016 - open.library.ubc.ca,… Figure 2.1 Specificity of two novel anti–Pan...,NaN,NaN,https://open.library.ubc.ca/media/download/pdf...,N121A/1,Pannexin-2


In [8]:
# get data file names - replace with your path for what i send
import glob
import pandas as pd

path = r'/Users/keithmitchell/Desktop/Repositories/NeuroMabSeq/trimmer/picklequeries/new_test2'
filenames = glob.glob(path + "/*.csv")

dfs = []
for filename in filenames:
    dfs.append(pd.read_csv(filename))
allnmab_df2 = pd.concat(dfs, ignore_index=True)

In [9]:
allnmab_df2

,title,title_link,publication_info,snippet,cited_by_link,cited_by_count,pdf_file,Antibody,Target
0,Deubiquitylating enzyme USP2 counteracts Nedd4...,https://www.sciencedirect.com/science/article/...,"K Krzystanek, HB Rasmussen, M Grunnet, O Staub...","BACKGROUND: KCNQ1 (Kv7.1), together with its K...",https://scholar.google.com/scholar?cites=98014...,35.0,NaN,N37A/10,Kv7.1/KCNQ1 K+ channel
1,Caracterización de los canales de potasio depe...,https://idus.us.es/handle/11441/138533,M Ortego Domínguez - 2022 - idus.us.es,Los adipocitos blancos son células que partici...,NaN,NaN,https://idus.us.es/bitstream/handle/11441/1385...,N375/67,Kv3.3 K+ channel
2,Neuroinflammation alters integrative propertie...,https://link.springer.com/article/10.1007/s120...,"F Frigerio, C Flynn, Y Han, K Lyman, JN Lugo… ...","… , N212/17, Neuromab, CA, USA) antibodies usi...",https://scholar.google.com/scholar?cites=19097...,41.0,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6...,N212/17,TRIP8b (constant)
3,Allostery between two binding sites in the io...,https://www.jbc.org/article/S0021-9258(20)3303...,"KA Lyman, Y Han, RJ Heuermann, X Cheng… - Jour...",Tetratricopeptide repeat (TPR) domains are ubi...,https://scholar.google.com/scholar?cites=14199...,13.0,https://www.jbc.org/article/S0021-9258(20)3303...,N212/17,TRIP8b (constant)
4,Distal dendritic enrichment of HCN1 channels i...,https://www.eneuro.org/content/5/5/eneuro.0258...,"M Meseke, F Neumüller, B Brunne, X Li, M Anstö...",HCN1 compartmentalization in CA1 pyramidal cel...,https://scholar.google.com/scholar?cites=50206...,9.0,https://www.eneuro.org/content/eneuro/5/5/ENEU...,N212/17,TRIP8b (constant)
...,...,...,...,...,...,...,...,...,...
3483,Protein components of the backbone structure o...,https://soar-ir.repo.nii.ac.jp/?action=reposit...,樋口京一， 竹下敏一 - soar-ir.repo.nii.ac.jp,(論文の内容の要旨)[背景と目的] シナプス後肥厚部 (postsynaptic densi...,NaN,NaN,https://soar-ir.repo.nii.ac.jp/?action=reposit...,N127/31,Pan-SAPAP
3484,Protein components of the backbone structure o...,https://soar-ir.repo.nii.ac.jp/?action=reposit...,郭維恒 - 2017 - soar-ir.repo.nii.ac.jp,(論文の内容の要旨)[背景と目的] シナプス後肥厚部 (postsynaptic densi...,NaN,NaN,https://soar-ir.repo.nii.ac.jp/record/19942/fi...,N127/31,Pan-SAPAP
3485,Pannexin 2 protein expression is not restrict...,https://www.frontiersin.org/articles/10.3389/f...,"M Le Vasseur, J Lelowski, JF Bechberger… - Fro...","… by UC Davis/NIH NeuroMab Facility (Davis, CA...",https://scholar.google.com/scholar?cites=77804...,84.0,https://www.frontiersin.org/articles/10.3389/f...,N121A/1,Pannexin-2
3486,The role of Pannexin 2 in mitochondrial functi...,https://open.library.ubc.ca/soa/cIRcle/collect...,M Le Vasseur - 2016 - open.library.ubc.ca,… Figure 2.1 Specificity of two novel anti–Pan...,NaN,NaN,https://open.library.ubc.ca/media/download/pdf...,N121A/1,Pannexin-2


In [10]:
# get data file names - replace with your path for what i send
import glob
import pandas as pd

path = r'/Users/keithmitchell/Desktop/Repositories/NeuroMabSeq/trimmer/picklequeries/new_test3'
filenames = glob.glob(path + "/*.csv")

dfs = []
for filename in filenames:
    dfs.append(pd.read_csv(filename))
allnmab_df3 = pd.concat(dfs, ignore_index=True)

In [11]:
allnmab_df3

,title,title_link,publication_info,snippet,cited_by_link,cited_by_count,pdf_file,Antibody,Target
0,Characterization of hyperpolarization-activate...,https://www.frontiersin.org/articles/10.3389/f...,"KA Lyman, Y Han, AP Robinson… - Frontiers in C...",Mature oligodendrocytes (OLG) are the myelin-f...,NaN,NaN,https://www.frontiersin.org/articles/10.3389/f...,N212/17,TRIP8b (constant)
1,A complex of Protocadherin-19 and N-cadherin m...,https://rupress.org/jcb/article-abstract/195/7...,"MR Emond, S Biswas, CJ Blevins, JD Jontes - Jo...",… N144/14; University of California Davis/Nati...,https://scholar.google.com/scholar?cites=11570...,126.0,https://rupress.org/jcb/article-pdf/195/7/1115...,N144/14,6xHis
2,Cdk5-mediated phosphorylation of CRMP-2 enhan...,https://www.sciencedirect.com/science/article/...,"JM Brittain, Y Wang, O Eruvwetere, R Khanna - ...","… Lysates were incubated with CRMP-2-6xHis, fo...",https://scholar.google.com/scholar?cites=16563...,84.0,https://www.sciencedirect.com/science/article/...,N144/14,6xHis
3,Reprogramming protein kinase substrate specifi...,https://www.biorxiv.org/content/10.1101/091892...,"JM Lubner, GM Church, MF Chou, D Schwartz - bi...",Protein kinase specificity is largely imparted...,https://scholar.google.com/scholar?cites=11711...,5.0,https://www.biorxiv.org/content/10.1101/091892...,N144/14,6xHis
4,α-synuclein promotes neuronal dysfunction and ...,https://www.jneurosci.org/content/43/9/1614.ab...,"G Maor, RR Dubreuil, MB Feany - Journal of Neu...",α-Synuclein plays a key role in the pathogenes...,https://scholar.google.com/scholar?cites=25347...,7.0,https://www.jneurosci.org/content/jneuro/43/9/...,N144/14,6xHis
...,...,...,...,...,...,...,...,...,...
1440,Dissecting The Tissue-Specific Contributions T...,https://scholar.smu.edu/hum_sci_biologicalscie...,K Paulhus - 2024 - scholar.smu.edu,"Sudden unexpected death in epilepsy (SUDEP), t...",NaN,NaN,https://scholar.smu.edu/cgi/viewcontent.cgi?ar...,K20/78,Kv1.1 K+ channel
1441,Impact of Elevated Brain IL-6 in Transgenic Mi...,https://www.mdpi.com/2073-4409/12/18/2306,"DL Gruol, D Calderon, S Huitron-Resendiz… - Ce...","… 500, UC Davis/NIH NeuroMab Facility); a mono...",https://scholar.google.com/scholar?cites=45817...,1.0,https://www.mdpi.com/2073-4409/12/18/2306/pdf,L118/80,VGAT
1442,Rescue of deficits by Brwd1 copy number restor...,https://www.nature.com/articles/s41467-022-342...,"SL Fulton, W Wenderski, AE Lepack, AL Eagle… -...","With an incidence of ~1 in 800 births, Down sy...",https://scholar.google.com/scholar?cites=44492...,9.0,https://www.nature.com/articles/s41467-022-342...,N336B/83,BAF53a
1443,Identification of Novel Cell-Type Specific Epi...,https://search.proquest.com/openview/88710f293...,SL Fulton - 2022 - search.proquest.com,The overarching goal of this thesis work has b...,NaN,NaN,https://search.proquest.com/openview/88710f293...,N336B/83,BAF53a


In [12]:
# merge the dataframes and drop duplicates
df_all = pd.concat([allnmab_df, allnmab_df2, allnmab_df3], ignore_index=True)
df_all

,title,title_link,publication_info,snippet,cited_by_link,cited_by_count,pdf_file,Antibody,Target
0,Deubiquitylating enzyme USP2 counteracts Nedd4...,https://www.sciencedirect.com/science/article/...,"K Krzystanek, HB Rasmussen, M Grunnet, O Staub...","BACKGROUND: KCNQ1 (Kv7.1), together with its K...",https://scholar.google.com/scholar?cites=98014...,35.0,NaN,N37A/10,Kv7.1/KCNQ1 K+ channel
1,Caracterización de los canales de potasio depe...,https://idus.us.es/handle/11441/138533,M Ortego Domínguez - 2022 - idus.us.es,Los adipocitos blancos son células que partici...,NaN,NaN,https://idus.us.es/bitstream/handle/11441/1385...,N375/67,Kv3.3 K+ channel
2,Neuroinflammation alters integrative propertie...,https://link.springer.com/article/10.1007/s120...,"F Frigerio, C Flynn, Y Han, K Lyman, JN Lugo… ...","… , N212/17, Neuromab, CA, USA) antibodies usi...",https://scholar.google.com/scholar?cites=19097...,41.0,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6...,N212/17,TRIP8b (constant)
3,Allostery between two binding sites in the io...,https://www.jbc.org/article/S0021-9258(20)3303...,"KA Lyman, Y Han, RJ Heuermann, X Cheng… - Jour...",Tetratricopeptide repeat (TPR) domains are ubi...,https://scholar.google.com/scholar?cites=14199...,12.0,https://www.jbc.org/article/S0021-9258(20)3303...,N212/17,TRIP8b (constant)
4,Distal dendritic enrichment of HCN1 channels i...,https://www.eneuro.org/content/5/5/eneuro.0258...,"M Meseke, F Neumüller, B Brunne, X Li, M Anstö...",HCN1 compartmentalization in CA1 pyramidal cel...,https://scholar.google.com/scholar?cites=50206...,8.0,https://www.eneuro.org/content/eneuro/5/5/ENEU...,N212/17,TRIP8b (constant)
...,...,...,...,...,...,...,...,...,...
8716,Dissecting The Tissue-Specific Contributions T...,https://scholar.smu.edu/hum_sci_biologicalscie...,K Paulhus - 2024 - scholar.smu.edu,"Sudden unexpected death in epilepsy (SUDEP), t...",NaN,NaN,https://scholar.smu.edu/cgi/viewcontent.cgi?ar...,K20/78,Kv1.1 K+ channel
8717,Impact of Elevated Brain IL-6 in Transgenic Mi...,https://www.mdpi.com/2073-4409/12/18/2306,"DL Gruol, D Calderon, S Huitron-Resendiz… - Ce...","… 500, UC Davis/NIH NeuroMab Facility); a mono...",https://scholar.google.com/scholar?cites=45817...,1.0,https://www.mdpi.com/2073-4409/12/18/2306/pdf,L118/80,VGAT
8718,Rescue of deficits by Brwd1 copy number restor...,https://www.nature.com/articles/s41467-022-342...,"SL Fulton, W Wenderski, AE Lepack, AL Eagle… -...","With an incidence of ~1 in 800 births, Down sy...",https://scholar.google.com/scholar?cites=44492...,9.0,https://www.nature.com/articles/s41467-022-342...,N336B/83,BAF53a
8719,Identification of Novel Cell-Type Specific Epi...,https://search.proquest.com/openview/88710f293...,SL Fulton - 2022 - search.proquest.com,The overarching goal of this thesis work has b...,NaN,NaN,https://search.proquest.com/openview/88710f293...,N336B/83,BAF53a


In [13]:
# groupby the title of the paper and create a new column that merges the antibody names but keep all the other columns
df_filter = df_all.groupby('title').agg({'Antibody': lambda x: ', '.join(set(x)), 
                                         'Target': lambda x: ', '.join(set(x))}).reset_index()
# enforce thaat all strings in the Antibody column are unique as well and then merge back to the original df
df_final = df_all.merge(df_filter, on='title', how='left').drop_duplicates(subset=['title'])
# drop the Antibody_y column and rename Antibody_x to Antibody
df_final = df_final.drop(['Antibody_x'], axis=1)
# same for the target column
df_final = df_final.drop(['Target_x'], axis=1)
df_final = df_final.rename(columns={'Target_y': 'Target'})
df_final = df_final.rename(columns={'Antibody_y': 'Antibody'})
df_final

,title,title_link,publication_info,snippet,cited_by_link,cited_by_count,pdf_file,Antibody,Target
0,Deubiquitylating enzyme USP2 counteracts Nedd4...,https://www.sciencedirect.com/science/article/...,"K Krzystanek, HB Rasmussen, M Grunnet, O Staub...","BACKGROUND: KCNQ1 (Kv7.1), together with its K...",https://scholar.google.com/scholar?cites=98014...,35.0,NaN,N37A/10,Kv7.1/KCNQ1 K+ channel
1,Caracterización de los canales de potasio depe...,https://idus.us.es/handle/11441/138533,M Ortego Domínguez - 2022 - idus.us.es,Los adipocitos blancos son células que partici...,NaN,NaN,https://idus.us.es/bitstream/handle/11441/1385...,"N375/67, L71/5, L23/27, K57/1","Kv1.4 K+ channel (extracellular), Kv1.3 K+ cha..."
2,Neuroinflammation alters integrative propertie...,https://link.springer.com/article/10.1007/s120...,"F Frigerio, C Flynn, Y Han, K Lyman, JN Lugo… ...","… , N212/17, Neuromab, CA, USA) antibodies usi...",https://scholar.google.com/scholar?cites=19097...,41.0,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6...,N212/17,TRIP8b (constant)
3,Allostery between two binding sites in the io...,https://www.jbc.org/article/S0021-9258(20)3303...,"KA Lyman, Y Han, RJ Heuermann, X Cheng… - Jour...",Tetratricopeptide repeat (TPR) domains are ubi...,https://scholar.google.com/scholar?cites=14199...,12.0,https://www.jbc.org/article/S0021-9258(20)3303...,N212/17,TRIP8b (constant)
4,Distal dendritic enrichment of HCN1 channels i...,https://www.eneuro.org/content/5/5/eneuro.0258...,"M Meseke, F Neumüller, B Brunne, X Li, M Anstö...",HCN1 compartmentalization in CA1 pyramidal cel...,https://scholar.google.com/scholar?cites=50206...,8.0,https://www.eneuro.org/content/eneuro/5/5/ENEU...,N212/17,TRIP8b (constant)
...,...,...,...,...,...,...,...,...,...
8705,Interleukin-6 secretion by astrocytes is dynam...,https://journals.plos.org/plosone/article?id=1...,"S Codeluppi, T Fernandez-Zafra, K Sandor, J Kj...","After contusion spinal cord injury (SCI), astr...",https://scholar.google.com/scholar?cites=10368...,48.0,https://journals.plos.org/plosone/article/file...,N103/31,Aldh1L1 (staining)
8706,Pathogenic mechanism of the FIG4 mutation resp...,https://journals.plos.org/plosgenetics/article...,"GM Lenk, CJ Ferguson, CY Chow, N Jin… - PLoS …...",… The monoclonal antibody was generated using ...,https://scholar.google.com/scholar?cites=12174...,129.0,https://journals.plos.org/plosgenetics/article...,N202/7,Fig4/Sac3
8716,Dissecting The Tissue-Specific Contributions T...,https://scholar.smu.edu/hum_sci_biologicalscie...,K Paulhus - 2024 - scholar.smu.edu,"Sudden unexpected death in epilepsy (SUDEP), t...",NaN,NaN,https://scholar.smu.edu/cgi/viewcontent.cgi?ar...,K20/78,Kv1.1 K+ channel
8718,Rescue of deficits by Brwd1 copy number restor...,https://www.nature.com/articles/s41467-022-342...,"SL Fulton, W Wenderski, AE Lepack, AL Eagle… -...","With an incidence of ~1 in 800 births, Down sy...",https://scholar.google.com/scholar?cites=44492...,9.0,https://www.nature.com/articles/s41467-022-342...,N336B/83,BAF53a


In [14]:
# filter df_final for Miyazaki
df_final[df_final['publication_info'].str.contains('Miyazaki')]

,title,title_link,publication_info,snippet,cited_by_link,cited_by_count,pdf_file,Antibody,Target
15,Chemical corrector treatment ameliorates incre...,https://www.nature.com/articles/nm.3759,"N Yokoi, Y Fukata, D Kase, T Miyazaki, M Jaegl...",Epilepsy is one of the most common and intract...,https://scholar.google.com/scholar?cites=35472...,74.0,NaN,"K14/16, N144/14","Kv1.2 K+ channel, 6xHis"
1831,Oligodendrocyte-derived LGI3 and its receptor...,https://www.cell.com/cell-reports/pdf/S2211-12...,"Y Miyazaki, T Otsuka, Y Yamagata, T Endo, M Sa...","Neurodevelopmental disorders, such as intellec...",NaN,NaN,https://www.cell.com/cell-reports/pdf/S2211-12...,"K13/31, L23/27, K67/25, K65/35, K14/16, K36/15...","CASPR2, ADAM22 (cytoplasmic), Kv1.4 K+ channel..."
1855,Proteomic analysis of native cerebellar iFGF14...,https://www.tandfonline.com/doi/abs/10.1080/19...,"MK Bosch, JM Nerbonne, RR Townsend, H Miyazaki...",Intracellular Fibroblast Growth Factor 14 (iFG...,https://scholar.google.com/scholar?cites=13497...,14.0,https://www.tandfonline.com/doi/pdf/10.1080/19...,"N56/21, K74/71, K69/3","Nav1.1 Na+ channel, Nav1.2 Na+ channel, FGF14/..."


In [4]:
# output the final df to the picklequeries folder as googescholar_mabidsearch.csv
allnmab_df3.to_csv('/Users/keithmitchell/Desktop/Repositories/NeuroMabSeq/trimmer/picklequeries/googescholar_mabidsearch_5-20-24.csv', index=False)

In [ ]:
# sum baased on the year


# Pubmed scraping get PMID for each of the papers



In [15]:
def add_Pubmed_info_to_dataframe(df, developer_mode, pos):
    # TODO wendon this function is a bit of a mess and could use some cleaning I 
        # i would just make sure it is working and make sure the "significance" checking is good.
        # then i would take a look at the "try" and "except" blocks and see if there is a better way to do it
        # might be a better way then just intermediary files but also a good way to cache and abstract - just dependent on the previous dataframe
        # solution could be reading in old outputs titles or something idk - just a thought surely something simple here
    '''
    Description : 
    
    This function uses the PubMed tool to generate the pubmed ID from the value in the Title 
    column of each row of the desired dataframe
    
    Input : 
    
    leftoff = from what row you would like to start adding information to 
    
    autosave = the number of iterations before being saved as csv
    
    df = the desired dataframe to add the information to 
    
    Returns : 
    
    a dataframe with the PMID, Author, Publication Date, and Journal columns appended
    
    '''

    # initialize the tool being used 
    pubmed = PubMed(tool="PubMedSearcher", email="gmail.com")

    # read in desired dataframe 
    #df = pd.read_csv(generate_path_to_merged_df())
    
    # Initialize columns values to be added to NaN
    if 'PMID' not in df:
        df['PMID'] = np.nan
    if 'Author' not in df:
        df['Author'] = np.nan
    if 'Publication Date' not in df:
        df['Publication Date'] = np.nan
    if 'Journal' not in df:
        df['Journal'] = np.nan

    # iterate over the df from the desginated row 
    for index, row in df[pos:].iterrows():
        try:
            if pd.isna(row['title']):    # if the value in Title is empty skip over it 
                continue

            if not pd.isna(row['PMID']): # if there is a value in the PMID column skip over it 
                continue

            search_term = row['title']
            max_results = 500
            results = pubmed.query(search_term, max_results=max_results)
            last_pubmedId = None  # Initialize to None

            # iterate through each of the paper titles in results, if the title is 80% similar to one of the 
            # articles - then that is the paper that we will grab information from 
            for article in results:
                title = article.title
                pubmedId = article.pubmed_id.partition('\n')[0]

                # Compare the similarity of the title and the search term
                similarity = fuzz.token_set_ratio(title.lower(), search_term.lower())
                if similarity >= 80:
                    last_pubmedId = pubmedId
                    break

            if last_pubmedId is not None:  # Only assign pubmed_id if a significant title is found
                print(row['title'],last_pubmedId)
                df.at[index, 'PMID'] = last_pubmedId   # assign the PMID to the row 
                df.at[index, 'Date'] = article.publication_date # assign the publication date to the row 

                # Create the Publication information from the first authors name and 
                # journal the paper is from 
                authors = article.authors
                # Extracting the first author's lastname
                first_author_lastname = authors[0]['lastname']

                # Creating the "et al" part
                et_al = "et al"

                # Creating the formatted string for publication info
                publication_info = f"{first_author_lastname} {et_al}. {article.journal}"
                df.at[index, 'Publication'] = publication_info
        
        except Exception as e:
            import traceback
            traceback.print_exc()
            print(e)
            df[pos:index].to_csv(f"/Users/keithmitchell/Desktop/Repositories/NeuroMabSeq/trimmer/picklequeries/merged_df_{pos}-{index}.csv", index=False)
            return df, index

        # save the dataframe whenever there are 100 iterations - there is a possibility of being dropped
    
    # remove the 'unnamed' column from the df 
    # for col in runresultsdf.columns:
    #     if col.startswith("Unnamed"):
    #         runresultsdf = runresultsdf.drop(col, axis=1)
            
    # Delete duplicates in the PMID column 
    #df = df.drop_duplicates(subset='PMID', keep='first')
    
    df.to_csv(f"/Users/keithmitchell/Desktop/Repositories/NeuroMabSeq/trimmer/picklequeries/merged_df_{pos}_{index}_done_5-20-24.csv", index=False)
    return df


In [17]:
# Add pubmed info to a dataframe
import os
import json
from Bio import Entrez

def fetch_details(pubmed_id):
    file_path = f"/Users/keithmitchell/Desktop/Repositories/NeuroMabSeq/trimmer/picklequeries/pmidjsons/{pubmed_id}.json"
    
    try:
        # Check if the file already exists to use cached data
        if os.path.exists(file_path):
            with open(file_path, 'r') as f:
                record = json.load(f)
        else:
            Entrez.email = "kgmitchell@ucdavis.edu"  # Always provide your email
            handle = Entrez.efetch(db="pubmed", id=pubmed_id, retmode="xml")
            records = Entrez.read(handle)
            handle.close()
            
            if not records or 'PubmedArticle' not in records:
                print(f"PubMed returned no data for {pubmed_id}.")
                return None

            record = records
            with open(file_path, 'w') as f:
                json.dump(record, f)

        article = record['PubmedArticle'][0]
        article_data = article['MedlineCitation']['Article']
        authors = article_data['AuthorList']
        
        # Use only the first author with "et al." regardless of the number of authors
        first_author = authors[0]
        author_names = f"{first_author['LastName']} {first_author['Initials']} et al."
        
        title = article_data['ArticleTitle']
        journal = article_data['Journal']['ISOAbbreviation']
        year = article_data['Journal']['JournalIssue']['PubDate']['Year']

        # Safely check for Pagination and Volume
        pages = ''
        if 'Pagination' in article_data:
            pages = article_data['Pagination'].get('MedlinePgn', '')

        volume = article_data['Journal']['JournalIssue'].get('Volume', '')

        # Construct citation based on available data
        citation_parts = [author_names, year, journal]
        if volume:
            citation_parts.append(volume)
        if pages:
            citation_parts.append(pages)

        citation = " ".join(citation_parts) + "."

        # Return the result as a dictionary
        result = {'citation': citation, 'title': title, 'year': year}
        return result
    except Exception as e:
        print(f"Error fetching data from PubMed for {pubmed_id}.")
        print(e)
        return None


In [22]:
df_final

,title,title_link,publication_info,snippet,cited_by_link,cited_by_count,pdf_file,Antibody,Target,PMID,Author,Publication Date,Journal,Date,Publication
0,Characterization of hyperpolarization-activate...,https://www.frontiersin.org/articles/10.3389/f...,"KA Lyman, Y Han, AP Robinson… - Frontiers in C...",Mature oligodendrocytes (OLG) are the myelin-f...,NaN,NaN,https://www.frontiersin.org/articles/10.3389/f...,N212/17,TRIP8b (constant),38469353,NaN,NaN,NaN,2024-03-12,Lyman et al. Frontiers in cellular neuroscience
1,A complex of Protocadherin-19 and N-cadherin m...,https://rupress.org/jcb/article-abstract/195/7...,"MR Emond, S Biswas, CJ Blevins, JD Jontes - Jo...",… N144/14; University of California Davis/Nati...,https://scholar.google.com/scholar?cites=11570...,126.0,https://rupress.org/jcb/article-pdf/195/7/1115...,N144/14,6xHis,22184198,NaN,NaN,NaN,2011-12-21,Emond et al. The Journal of cell biology
2,Cdk5-mediated phosphorylation of CRMP-2 enhan...,https://www.sciencedirect.com/science/article/...,"JM Brittain, Y Wang, O Eruvwetere, R Khanna - ...","… Lysates were incubated with CRMP-2-6xHis, fo...",https://scholar.google.com/scholar?cites=16563...,84.0,https://www.sciencedirect.com/science/article/...,N144/14,6xHis,23022559,NaN,NaN,NaN,2012-10-02,Brittain et al. FEBS letters
3,Reprogramming protein kinase substrate specifi...,https://www.biorxiv.org/content/10.1101/091892...,"JM Lubner, GM Church, MF Chou, D Schwartz - bi...",Protein kinase specificity is largely imparted...,https://scholar.google.com/scholar?cites=11711...,5.0,https://www.biorxiv.org/content/10.1101/091892...,N144/14,6xHis,NaN,NaN,NaN,NaN,NaN,NaN
4,α-synuclein promotes neuronal dysfunction and ...,https://www.jneurosci.org/content/43/9/1614.ab...,"G Maor, RR Dubreuil, MB Feany - Journal of Neu...",α-Synuclein plays a key role in the pathogenes...,https://scholar.google.com/scholar?cites=25347...,7.0,https://www.jneurosci.org/content/jneuro/43/9/...,N144/14,6xHis,37333277,NaN,NaN,NaN,2023-06-19,Maor et al. bioRxiv : the preprint server for ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1440,Dissecting The Tissue-Specific Contributions T...,https://scholar.smu.edu/hum_sci_biologicalscie...,K Paulhus - 2024 - scholar.smu.edu,"Sudden unexpected death in epilepsy (SUDEP), t...",NaN,NaN,https://scholar.smu.edu/cgi/viewcontent.cgi?ar...,K20/78,Kv1.1 K+ channel,NaN,NaN,NaN,NaN,NaN,NaN
1441,Impact of Elevated Brain IL-6 in Transgenic Mi...,https://www.mdpi.com/2073-4409/12/18/2306,"DL Gruol, D Calderon, S Huitron-Resendiz… - Ce...","… 500, UC Davis/NIH NeuroMab Facility); a mono...",https://scholar.google.com/scholar?cites=45817...,1.0,https://www.mdpi.com/2073-4409/12/18/2306/pdf,L118/80,VGAT,NaN,NaN,NaN,NaN,NaN,NaN
1442,Rescue of deficits by Brwd1 copy number restor...,https://www.nature.com/articles/s41467-022-342...,"SL Fulton, W Wenderski, AE Lepack, AL Eagle… -...","With an incidence of ~1 in 800 births, Down sy...",https://scholar.google.com/scholar?cites=44492...,9.0,https://www.nature.com/articles/s41467-022-342...,N336B/83,BAF53a,NaN,NaN,NaN,NaN,NaN,NaN
1443,Identification of Novel Cell-Type Specific Epi...,https://search.proquest.com/openview/88710f293...,SL Fulton - 2022 - search.proquest.com,The overarching goal of this thesis work has b...,NaN,NaN,https://search.proquest.com/openview/88710f293...,N336B/83,BAF53a,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
pos = 0
df_final = pd.read_csv('/Users/keithmitchell/Desktop/Repositories/NeuroMabSeq/trimmer/picklequeries/googescholar_mabidsearch_5-20-24.csv')
while pos < len(df_final):
    print(pos)
    result = add_Pubmed_info_to_dataframe(df_final, False, pos)
    pos = result[1]

0
Characterization of hyperpolarization-activated cyclic nucleotide-gated channels in oligodendrocytes 38469353
A complex of Protocadherin-19 and N-cadherin mediates a novel mechanism of cell adhesion 22184198
 Cdk5-mediated phosphorylation of CRMP-2 enhances its interaction with CaV2. 2 23022559
α-synuclein promotes neuronal dysfunction and death by disrupting the binding of ankyrin to β-spectrin 37333277
 Direct binding between pre-S1 and TRP-like domains in TRPP channels mediates gating and functional regulation by PIP2 29425510
A cytoskeleton-membrane interaction conserved in fast-spiking neurons controls movement, emotion, and memory 37833406
Structural basis for recognition of synaptic vesicle protein 2C by botulinum neurotoxin A 24240280
 The metalloprotease, Mpr1, engages AnnexinA2 to promote the transcytosis of fungal cells across the blood-brain barrier 28713781
Activation of conventional kinesin motors in clusters by Shaw voltage-gated K+ channels 23487040
β spectrin-depende

KeyboardInterrupt: 

In [11]:
result

,title,title_link,publication_info,snippet,cited_by_link,cited_by_count,pdf_file,Antibody,Target,PMID,Author,Publication Date,Journal,Date,Publication
0,Characterization of hyperpolarization-activate...,https://www.frontiersin.org/articles/10.3389/f...,"KA Lyman, Y Han, AP Robinson… - Frontiers in C...",Mature oligodendrocytes (OLG) are the myelin-f...,NaN,NaN,https://www.frontiersin.org/articles/10.3389/f...,N212/17,TRIP8b (constant),38469353,NaN,NaN,NaN,2024-03-12,Lyman et al. Frontiers in cellular neuroscience
1,A complex of Protocadherin-19 and N-cadherin m...,https://rupress.org/jcb/article-abstract/195/7...,"MR Emond, S Biswas, CJ Blevins, JD Jontes - Jo...",… N144/14; University of California Davis/Nati...,https://scholar.google.com/scholar?cites=11570...,126.0,https://rupress.org/jcb/article-pdf/195/7/1115...,N144/14,6xHis,22184198,NaN,NaN,NaN,2011-12-21,Emond et al. The Journal of cell biology
2,Cdk5-mediated phosphorylation of CRMP-2 enhan...,https://www.sciencedirect.com/science/article/...,"JM Brittain, Y Wang, O Eruvwetere, R Khanna - ...","… Lysates were incubated with CRMP-2-6xHis, fo...",https://scholar.google.com/scholar?cites=16563...,84.0,https://www.sciencedirect.com/science/article/...,N144/14,6xHis,23022559,NaN,NaN,NaN,2012-10-02,Brittain et al. FEBS letters
3,Reprogramming protein kinase substrate specifi...,https://www.biorxiv.org/content/10.1101/091892...,"JM Lubner, GM Church, MF Chou, D Schwartz - bi...",Protein kinase specificity is largely imparted...,https://scholar.google.com/scholar?cites=11711...,5.0,https://www.biorxiv.org/content/10.1101/091892...,N144/14,6xHis,NaN,NaN,NaN,NaN,NaN,NaN
4,α-synuclein promotes neuronal dysfunction and ...,https://www.jneurosci.org/content/43/9/1614.ab...,"G Maor, RR Dubreuil, MB Feany - Journal of Neu...",α-Synuclein plays a key role in the pathogenes...,https://scholar.google.com/scholar?cites=25347...,7.0,https://www.jneurosci.org/content/jneuro/43/9/...,N144/14,6xHis,37333277,NaN,NaN,NaN,2023-06-19,Maor et al. bioRxiv : the preprint server for ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1440,Dissecting The Tissue-Specific Contributions T...,https://scholar.smu.edu/hum_sci_biologicalscie...,K Paulhus - 2024 - scholar.smu.edu,"Sudden unexpected death in epilepsy (SUDEP), t...",NaN,NaN,https://scholar.smu.edu/cgi/viewcontent.cgi?ar...,K20/78,Kv1.1 K+ channel,NaN,NaN,NaN,NaN,NaN,NaN
1441,Impact of Elevated Brain IL-6 in Transgenic Mi...,https://www.mdpi.com/2073-4409/12/18/2306,"DL Gruol, D Calderon, S Huitron-Resendiz… - Ce...","… 500, UC Davis/NIH NeuroMab Facility); a mono...",https://scholar.google.com/scholar?cites=45817...,1.0,https://www.mdpi.com/2073-4409/12/18/2306/pdf,L118/80,VGAT,37759527,NaN,NaN,NaN,2023-09-28,Gruol et al. Cells
1442,Rescue of deficits by Brwd1 copy number restor...,https://www.nature.com/articles/s41467-022-342...,"SL Fulton, W Wenderski, AE Lepack, AL Eagle… -...","With an incidence of ~1 in 800 births, Down sy...",https://scholar.google.com/scholar?cites=44492...,9.0,https://www.nature.com/articles/s41467-022-342...,N336B/83,BAF53a,36289231,NaN,NaN,NaN,2022-10-27,Fulton et al. Nature communications
1443,Identification of Novel Cell-Type Specific Epi...,https://search.proquest.com/openview/88710f293...,SL Fulton - 2022 - search.proquest.com,The overarching goal of this thesis work has b...,NaN,NaN,https://search.proquest.com/openview/88710f293...,N336B/83,BAF53a,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
# read all the merged_df file sin the directory and concat them together
import glob
import pandas as pd

path = r'/Users/keithmitchell/Desktop/Repositories/NeuroMabSeq/trimmer/picklequeries'
filenames = glob.glob(path + "/merged_df_*.csv")
 
dfs = []
for file in filenames:
    dfs.append(pd.read_csv(file))

df_finaln = pd.concat(dfs, ignore_index=True)
# drop duplicaates based on the pubmed id, antibody and title
df_finaln = df_finaln.drop_duplicates(subset=['PMID', 'Antibody', 'title'])
df_finaln

,title,title_link,publication_info,snippet,cited_by_link,cited_by_count,pdf_file,Antibody,Target,PMID,Author,Publication Date,Journal,Date,Publication
0,Deubiquitylating enzyme USP2 counteracts Nedd4...,https://www.sciencedirect.com/science/article/...,"K Krzystanek, HB Rasmussen, M Grunnet, O Staub...","BACKGROUND: KCNQ1 (Kv7.1), together with its K...",https://scholar.google.com/scholar?cites=98014...,35.0,NaN,N37A/10,Kv7.1/KCNQ1 K+ channel,22024150.0,NaN,NaN,NaN,2011-10-26,Krzystanek et al. Heart rhythm
1,Caracterización de los canales de potasio depe...,https://idus.us.es/handle/11441/138533,M Ortego Domínguez - 2022 - idus.us.es,Los adipocitos blancos son células que partici...,NaN,NaN,https://idus.us.es/bitstream/handle/11441/1385...,N375/67,Kv3.3 K+ channel,NaN,NaN,NaN,NaN,NaN,NaN
2,Neuroinflammation alters integrative propertie...,https://link.springer.com/article/10.1007/s120...,"F Frigerio, C Flynn, Y Han, K Lyman, JN Lugo… ...","… , N212/17, Neuromab, CA, USA) antibodies usi...",https://scholar.google.com/scholar?cites=19097...,41.0,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6...,N212/17,TRIP8b (constant),29427087.0,NaN,NaN,NaN,2018-02-11,Frigerio et al. Molecular neurobiology
3,Allostery between two binding sites in the io...,https://www.jbc.org/article/S0021-9258(20)3303...,"KA Lyman, Y Han, RJ Heuermann, X Cheng… - Jour...",Tetratricopeptide repeat (TPR) domains are ubi...,https://scholar.google.com/scholar?cites=14199...,12.0,https://www.jbc.org/article/S0021-9258(20)3303...,N212/17,TRIP8b (constant),28887304.0,NaN,NaN,NaN,2017-09-10,Lyman et al. The Journal of biological chemistry
4,Distal dendritic enrichment of HCN1 channels i...,https://www.eneuro.org/content/5/5/eneuro.0258...,"M Meseke, F Neumüller, B Brunne, X Li, M Anstö...",HCN1 compartmentalization in CA1 pyramidal cel...,https://scholar.google.com/scholar?cites=50206...,8.0,https://www.eneuro.org/content/eneuro/5/5/ENEU...,N212/17,TRIP8b (constant),NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13373,A dominant dendrite phenotype caused by the di...,https://www.jbc.org/article/S0021-9258(20)3112...,"CC Yap, L Digilio, K Kruczek, M Roszkowska… - ...",Doublecortin (DCX) is a protein needed for cor...,https://scholar.google.com/scholar?cites=10171...,9.0,https://www.jbc.org/article/S0021-9258(20)3112...,A12/18,Pan-Neurofascin (extracellular),NaN,NaN,NaN,NaN,NaN,NaN
13374,RUFY3 and RUFY4 are ARL8 effectors that promot...,https://www.nature.com/articles/s41467-022-289...,"T Keren-Kaplan, A Sarić, S Ghosh… - Nature …, ...",The small GTPase ARL8 associates with endolyso...,https://scholar.google.com/scholar?cites=13157...,26.0,https://www.nature.com/articles/s41467-022-289...,A12/18,Pan-Neurofascin (extracellular),NaN,NaN,NaN,NaN,NaN,NaN
13375,Reversible association with motor proteins (RA...,https://journals.plos.org/plosbiology/article?...,"CM Guardia, R De Pace, A Sen, A Saric, M Jarni...",We report the development and characterization...,https://scholar.google.com/scholar?cites=72248...,24.0,https://journals.plos.org/plosbiology/article/...,A12/18,Pan-Neurofascin (extracellular),NaN,NaN,NaN,NaN,NaN,NaN
13377,Differential Alteration in Expression of Stri...,https://www.academia.edu/download/90605829/d09...,"Z Du, M Tertrais, G Courtand, T Leste-Lasserre...",Huntington’s disease (HD) is a neurodegenerati...,NaN,NaN,https://www.academia.edu/download/90605829/d09...,N87/25,"GABA(A)R, Beta3",NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
#filter df_finaln for 38194969
df_finaln[df_finaln['PMID'] == 38194969]


,title,title_link,publication_info,snippet,cited_by_link,cited_by_count,pdf_file,Antibody,Target,PMID,Author,Publication Date,Journal,Date,Publication
1797,Oligodendrocyte-derived LGI3 and its receptor...,https://www.cell.com/cell-reports/pdf/S2211-12...,"Y Miyazaki, T Otsuka, Y Yamagata, T Endo, M Sa...","Neurodevelopmental disorders, such as intellec...",https://scholar.google.com/scholar?cites=40492...,1.0,https://www.cell.com/cell-reports/pdf/S2211-12...,K36/15,Kv1.1 K+ channel (external),38194969.0,NaN,NaN,NaN,2024-01-10,Miyazaki et al. Cell reports
1867,Oligodendrocyte-derived LGI3 and its receptor...,https://www.cell.com/cell-reports/pdf/S2211-12...,"Y Miyazaki, T Otsuka, Y Yamagata, T Endo, M Sa...","Neurodevelopmental disorders, such as intellec...",https://scholar.google.com/scholar?cites=40492...,1.0,https://www.cell.com/cell-reports/pdf/S2211-12...,K13/31,Kv1.4 K+ channel,38194969.0,NaN,NaN,NaN,2024-01-10,Miyazaki et al. Cell reports
1909,Oligodendrocyte-derived LGI3 and its receptor...,https://www.cell.com/cell-reports/pdf/S2211-12...,"Y Miyazaki, T Otsuka, Y Yamagata, T Endo, M Sa...","Neurodevelopmental disorders, such as intellec...",https://scholar.google.com/scholar?cites=40492...,1.0,https://www.cell.com/cell-reports/pdf/S2211-12...,L23/27,Kv1.3 K+ channel,38194969.0,NaN,NaN,NaN,2024-01-10,Miyazaki et al. Cell reports
1939,Oligodendrocyte-derived LGI3 and its receptor...,https://www.cell.com/cell-reports/pdf/S2211-12...,"Y Miyazaki, T Otsuka, Y Yamagata, T Endo, M Sa...","Neurodevelopmental disorders, such as intellec...",https://scholar.google.com/scholar?cites=40492...,1.0,https://www.cell.com/cell-reports/pdf/S2211-12...,K14/16,Kv1.2 K+ channel,38194969.0,NaN,NaN,NaN,2024-01-10,Miyazaki et al. Cell reports
1966,Oligodendrocyte-derived LGI3 and its receptor...,https://www.cell.com/cell-reports/pdf/S2211-12...,"Y Miyazaki, T Otsuka, Y Yamagata, T Endo, M Sa...","Neurodevelopmental disorders, such as intellec...",https://scholar.google.com/scholar?cites=40492...,1.0,https://www.cell.com/cell-reports/pdf/S2211-12...,N46/30,ADAM22 (cytoplasmic),38194969.0,NaN,NaN,NaN,2024-01-10,Miyazaki et al. Cell reports
2129,Oligodendrocyte-derived LGI3 and its receptor...,https://www.cell.com/cell-reports/pdf/S2211-12...,"Y Miyazaki, T Otsuka, Y Yamagata, T Endo, M Sa...","Neurodevelopmental disorders, such as intellec...",https://scholar.google.com/scholar?cites=40492...,1.0,https://www.cell.com/cell-reports/pdf/S2211-12...,K65/35,CASPR/Neurexin IV,38194969.0,NaN,NaN,NaN,2024-01-10,Miyazaki et al. Cell reports
2164,Oligodendrocyte-derived LGI3 and its receptor...,https://www.cell.com/cell-reports/pdf/S2211-12...,"Y Miyazaki, T Otsuka, Y Yamagata, T Endo, M Sa...","Neurodevelopmental disorders, such as intellec...",https://scholar.google.com/scholar?cites=40492...,1.0,https://www.cell.com/cell-reports/pdf/S2211-12...,K67/25,CASPR2,38194969.0,NaN,NaN,NaN,2024-01-10,Miyazaki et al. Cell reports
12124,Oligodendrocyte-derived LGI3 and its receptor...,https://www.cell.com/cell-reports/pdf/S2211-12...,"Y Miyazaki, T Otsuka, Y Yamagata, T Endo, M Sa...","Neurodevelopmental disorders, such as intellec...",NaN,NaN,https://www.cell.com/cell-reports/pdf/S2211-12...,"N46/30, K36/15, K67/25, L23/27, K13/31, K65/35",ADAM22 (cytoplasmic),38194969.0,NaN,NaN,NaN,2024-01-10,Miyazaki et al. Cell reports


In [14]:
# filte to only things with a pubmed id
df_finaln = df_finaln[df_finaln['PMID'].notna()]
df_finaln

,title,title_link,publication_info,snippet,cited_by_link,cited_by_count,pdf_file,Antibody,Target,PMID,Author,Publication Date,Journal,Date,Publication
0,Deubiquitylating enzyme USP2 counteracts Nedd4...,https://www.sciencedirect.com/science/article/...,"K Krzystanek, HB Rasmussen, M Grunnet, O Staub...","BACKGROUND: KCNQ1 (Kv7.1), together with its K...",https://scholar.google.com/scholar?cites=98014...,35.0,NaN,N37A/10,Kv7.1/KCNQ1 K+ channel,22024150.0,NaN,NaN,NaN,2011-10-26,Krzystanek et al. Heart rhythm
2,Neuroinflammation alters integrative propertie...,https://link.springer.com/article/10.1007/s120...,"F Frigerio, C Flynn, Y Han, K Lyman, JN Lugo… ...","… , N212/17, Neuromab, CA, USA) antibodies usi...",https://scholar.google.com/scholar?cites=19097...,41.0,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6...,N212/17,TRIP8b (constant),29427087.0,NaN,NaN,NaN,2018-02-11,Frigerio et al. Molecular neurobiology
3,Allostery between two binding sites in the io...,https://www.jbc.org/article/S0021-9258(20)3303...,"KA Lyman, Y Han, RJ Heuermann, X Cheng… - Jour...",Tetratricopeptide repeat (TPR) domains are ubi...,https://scholar.google.com/scholar?cites=14199...,12.0,https://www.jbc.org/article/S0021-9258(20)3303...,N212/17,TRIP8b (constant),28887304.0,NaN,NaN,NaN,2017-09-10,Lyman et al. The Journal of biological chemistry
5,The TRIP8b-HCN Interaction as a Therapeutic Ta...,https://search.proquest.com/openview/493f089f1...,KA Lyman - 2016 - search.proquest.com,Hyperpolarization-activated cyclic nucleotide-...,NaN,NaN,NaN,N212/17,TRIP8b (constant),28127990.0,NaN,NaN,NaN,2017-01-28,Lyman et al. Expert opinion on therapeutic tar...
6,A complex of Protocadherin-19 and N-cadherin m...,https://rupress.org/jcb/article-abstract/195/7...,"MR Emond, S Biswas, CJ Blevins, JD Jontes - Jo...",… N144/14; University of California Davis/Nati...,https://scholar.google.com/scholar?cites=11570...,116.0,https://rupress.org/jcb/article/195/7/1115/50544,N144/14,6xHis,22184198.0,NaN,NaN,NaN,2011-12-21,Emond et al. The Journal of cell biology
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13007,Kv4 channels underlie A-currents with highly v...,https://link.springer.com/article/10.1007/s004...,"C Strube, L Saliba, E Moubarak, V Penalba… - …...",… Immunodetections of K v 4.2 and K v 4.3 chan...,https://scholar.google.com/scholar?cites=66496...,15.0,NaN,"K57/1, K75/41",Kv4.3 K+ channel,24872163.0,NaN,NaN,NaN,2014-05-30,Strube et al. Pflugers Archiv : European journ...
13010,"Expression, cellular and subcellular localisat...",https://www.mdpi.com/1422-0067/20/2/246,"R Alfaro-Ruíz, C Aguado, A Martín-Belmonte… - ...","… -S2 loop, Q9NZV8; NeuroMab, UC Davis/NIH, Da...",https://scholar.google.com/scholar?cites=84227...,23.0,https://www.mdpi.com/1422-0067/20/2/246/htm,"K57/1, K75/41",Kv4.3 K+ channel,30634540.0,NaN,NaN,NaN,2019-01-13,Alfaro-Ruíz et al. International journal of mo...
13013,DPP6 localization in brain supports function ...,https://www.frontiersin.org/articles/10.3389/n...,"BD Clark, E Kwon, J Maffie, HY Jeong… - Fronti...",… K75/41 Neuromab; www.neuromab.org were used ...,https://scholar.google.com/scholar?cites=15195...,81.0,https://www.frontiersin.org/articles/10.3389/n...,"K57/1, K75/41",Kv4.3 K+ channel,18978958.0,NaN,NaN,NaN,2008-11-04,Clark et al. Frontiers in molecular neuroscience
13020,Erbin interacts with TARP γ-2 for surface expr...,https://www.nature.com/articles/nn.3320,"Y Tao, YJ Chen, C Shen, Z Luo, CR Bates, D Lee...",Inhibitory neurons control the firing of gluta...,https://scholar.google.com/scholar?cites=11307...,51.0,https://www.researchgate.net/profile/Tian-Ming...,"N245/36, N245/1",TARPGamma2/Stargazin,23354328.0,NaN,NaN,NaN,2013-01-29,Tao et al. Nature neuroscience


In [15]:
# output that to a csv file
df_finaln.to_csv("/Users/keithmitchell/Desktop/Repositories/NeuroMabSeq/trimmer/picklequeries/googlescholar_mabidsearch_pubmed_5-20-24.csv", index=False)

# Scrape the antibodies Inc Pages for all the entities to check.

In [16]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
from selenium import webdriver
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import re
import os

def scrape_matching_links(base_url, num_items, items_per_page, url_pattern, output_dir, conj):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    total_pages = (num_items // items_per_page) + (1 if num_items % items_per_page else 0)
    options = Options()
    options.headless = True
    browser = webdriver.Chrome(options=options)

    for page in range(1, total_pages + 1):
        output_file = f"{output_dir}/page_{page}_{conj}.csv"
        if os.path.exists(output_file):
            print(f"File {output_file} already exists. Skipping page {page}.")
            continue

        print(f"Processing page {page}...")
        url = f"{base_url}&page={page}"
        browser.get(url)
        html = browser.page_source

        if not html or "No results found" in html:
            print("Empty response or no results found. Terminating script.")
            browser.quit()
            return

        soup = BeautifulSoup(html, 'html.parser')
        links = [f"https://www.antibodiesinc.com{link['href']}" for link in soup.find_all('a', href=True) if url_pattern in link['href']]

        if not links:
            print("No links found on this page. Terminating script.")
            browser.quit()
            return

        df = pd.DataFrame(links, columns=['Link'])
        df.to_csv(output_file, index=False)
        print(f"Saved {output_file}")

    browser.quit()

# Modify these parameters according to your needs
#conj_list = ['490', 'Unconjugated', "550", "594", "650"]
conj_list = ['490', 'Unconjugated', "550", "594", "650"]

for conj in conj_list:
    base_url = f"https://www.antibodiesinc.com/search?refinementList%5Bvendor%5D%5B0%5D=NeuroMab&refinementList%5Bnamed_tags.conj%5D%5B0%5D={conj}"
    num_items = 1000
    items_per_page = 15
    url_pattern = "products"
    output_dir = "/Users/keithmitchell/Desktop/Repositories/NeuroMabSeq/trimmer/picklequeries/antibodiesinclinks/pages/"

    scrape_matching_links(base_url, num_items, items_per_page, url_pattern, output_dir, conj)


<ipython-input-16-fa80db5e0aff>:21: DeprecationWarning: headless property is deprecated, instead use add_argument('--headless') or add_argument('--headless=new')
  options.headless = True


File /Users/keithmitchell/Desktop/Repositories/NeuroMabSeq/trimmer/picklequeries/antibodiesinclinks/pages//page_1_490.csv already exists. Skipping page 1.
File /Users/keithmitchell/Desktop/Repositories/NeuroMabSeq/trimmer/picklequeries/antibodiesinclinks/pages//page_2_490.csv already exists. Skipping page 2.
File /Users/keithmitchell/Desktop/Repositories/NeuroMabSeq/trimmer/picklequeries/antibodiesinclinks/pages//page_3_490.csv already exists. Skipping page 3.
File /Users/keithmitchell/Desktop/Repositories/NeuroMabSeq/trimmer/picklequeries/antibodiesinclinks/pages//page_4_490.csv already exists. Skipping page 4.
File /Users/keithmitchell/Desktop/Repositories/NeuroMabSeq/trimmer/picklequeries/antibodiesinclinks/pages//page_5_490.csv already exists. Skipping page 5.
File /Users/keithmitchell/Desktop/Repositories/NeuroMabSeq/trimmer/picklequeries/antibodiesinclinks/pages//page_6_490.csv already exists. Skipping page 6.
File /Users/keithmitchell/Desktop/Repositories/NeuroMabSeq/trimmer/pic

<ipython-input-16-fa80db5e0aff>:21: DeprecationWarning: headless property is deprecated, instead use add_argument('--headless') or add_argument('--headless=new')
  options.headless = True


File /Users/keithmitchell/Desktop/Repositories/NeuroMabSeq/trimmer/picklequeries/antibodiesinclinks/pages//page_1_Unconjugated.csv already exists. Skipping page 1.
File /Users/keithmitchell/Desktop/Repositories/NeuroMabSeq/trimmer/picklequeries/antibodiesinclinks/pages//page_2_Unconjugated.csv already exists. Skipping page 2.
File /Users/keithmitchell/Desktop/Repositories/NeuroMabSeq/trimmer/picklequeries/antibodiesinclinks/pages//page_3_Unconjugated.csv already exists. Skipping page 3.
File /Users/keithmitchell/Desktop/Repositories/NeuroMabSeq/trimmer/picklequeries/antibodiesinclinks/pages//page_4_Unconjugated.csv already exists. Skipping page 4.
File /Users/keithmitchell/Desktop/Repositories/NeuroMabSeq/trimmer/picklequeries/antibodiesinclinks/pages//page_5_Unconjugated.csv already exists. Skipping page 5.
File /Users/keithmitchell/Desktop/Repositories/NeuroMabSeq/trimmer/picklequeries/antibodiesinclinks/pages//page_6_Unconjugated.csv already exists. Skipping page 6.
File /Users/keit

<ipython-input-16-fa80db5e0aff>:21: DeprecationWarning: headless property is deprecated, instead use add_argument('--headless') or add_argument('--headless=new')
  options.headless = True


File /Users/keithmitchell/Desktop/Repositories/NeuroMabSeq/trimmer/picklequeries/antibodiesinclinks/pages//page_1_550.csv already exists. Skipping page 1.
File /Users/keithmitchell/Desktop/Repositories/NeuroMabSeq/trimmer/picklequeries/antibodiesinclinks/pages//page_2_550.csv already exists. Skipping page 2.
File /Users/keithmitchell/Desktop/Repositories/NeuroMabSeq/trimmer/picklequeries/antibodiesinclinks/pages//page_3_550.csv already exists. Skipping page 3.
File /Users/keithmitchell/Desktop/Repositories/NeuroMabSeq/trimmer/picklequeries/antibodiesinclinks/pages//page_4_550.csv already exists. Skipping page 4.
File /Users/keithmitchell/Desktop/Repositories/NeuroMabSeq/trimmer/picklequeries/antibodiesinclinks/pages//page_5_550.csv already exists. Skipping page 5.
File /Users/keithmitchell/Desktop/Repositories/NeuroMabSeq/trimmer/picklequeries/antibodiesinclinks/pages//page_6_550.csv already exists. Skipping page 6.
File /Users/keithmitchell/Desktop/Repositories/NeuroMabSeq/trimmer/pic

<ipython-input-16-fa80db5e0aff>:21: DeprecationWarning: headless property is deprecated, instead use add_argument('--headless') or add_argument('--headless=new')
  options.headless = True


File /Users/keithmitchell/Desktop/Repositories/NeuroMabSeq/trimmer/picklequeries/antibodiesinclinks/pages//page_1_594.csv already exists. Skipping page 1.
File /Users/keithmitchell/Desktop/Repositories/NeuroMabSeq/trimmer/picklequeries/antibodiesinclinks/pages//page_2_594.csv already exists. Skipping page 2.
File /Users/keithmitchell/Desktop/Repositories/NeuroMabSeq/trimmer/picklequeries/antibodiesinclinks/pages//page_3_594.csv already exists. Skipping page 3.
File /Users/keithmitchell/Desktop/Repositories/NeuroMabSeq/trimmer/picklequeries/antibodiesinclinks/pages//page_4_594.csv already exists. Skipping page 4.
File /Users/keithmitchell/Desktop/Repositories/NeuroMabSeq/trimmer/picklequeries/antibodiesinclinks/pages//page_5_594.csv already exists. Skipping page 5.
File /Users/keithmitchell/Desktop/Repositories/NeuroMabSeq/trimmer/picklequeries/antibodiesinclinks/pages//page_6_594.csv already exists. Skipping page 6.
File /Users/keithmitchell/Desktop/Repositories/NeuroMabSeq/trimmer/pic

<ipython-input-16-fa80db5e0aff>:21: DeprecationWarning: headless property is deprecated, instead use add_argument('--headless') or add_argument('--headless=new')
  options.headless = True


File /Users/keithmitchell/Desktop/Repositories/NeuroMabSeq/trimmer/picklequeries/antibodiesinclinks/pages//page_1_650.csv already exists. Skipping page 1.
File /Users/keithmitchell/Desktop/Repositories/NeuroMabSeq/trimmer/picklequeries/antibodiesinclinks/pages//page_2_650.csv already exists. Skipping page 2.
File /Users/keithmitchell/Desktop/Repositories/NeuroMabSeq/trimmer/picklequeries/antibodiesinclinks/pages//page_3_650.csv already exists. Skipping page 3.
File /Users/keithmitchell/Desktop/Repositories/NeuroMabSeq/trimmer/picklequeries/antibodiesinclinks/pages//page_4_650.csv already exists. Skipping page 4.
File /Users/keithmitchell/Desktop/Repositories/NeuroMabSeq/trimmer/picklequeries/antibodiesinclinks/pages//page_5_650.csv already exists. Skipping page 5.
File /Users/keithmitchell/Desktop/Repositories/NeuroMabSeq/trimmer/picklequeries/antibodiesinclinks/pages//page_6_650.csv already exists. Skipping page 6.
File /Users/keithmitchell/Desktop/Repositories/NeuroMabSeq/trimmer/pic

In [24]:
# create a master dataframe of all the pmids so we can track the index
df = pd.DataFrame()
for file in glob.glob("/Users/keithmitchell/Desktop/Repositories/NeuroMabSeq/trimmer/picklequeries/antibodiesinclinks/pages/*.csv"):
    dfi = pd.read_csv(file)
    df = df.append(dfi)
    df = df.reset_index(drop=True)
# output this file
df.to_csv("/Users/keithmitchell/Desktop/Repositories/NeuroMabSeq/trimmer/picklequeries/antibodiesinclinks/antibodiesinc_master-5-20-24.csv", index=False)

<ipython-input-24-68d128074f2d>:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(dfi)
<ipython-input-24-68d128074f2d>:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(dfi)
<ipython-input-24-68d128074f2d>:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(dfi)
<ipython-input-24-68d128074f2d>:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(dfi)
<ipython-input-24-68d128074f2d>:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(dfi)
<ipython-input-24-68d128074f2d>:5: FutureWarning: 

# Now we need to get the citations from antibodies inc for each of these.

In [25]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import re
import os
import time
import random

def extract_pmids(urls, base_output_dir, start_index, overwrite):
    output_dir = os.path.join(base_output_dir, 'pmid_output')
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    options = Options()
    options.headless = True
    options.add_argument("window-size=1920x1080")

    user_agents = [
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
        # Add more user agents here
    ]

    browser = webdriver.Chrome(options=options)
    number_of_0s = 170
    number_of_0s_count = 0
    # start at the index provided 
    for index, url in enumerate(urls[start_index:]):
        print(f"Processing URL {url}...")
        # check if file already exists
        output_file = os.path.join(output_dir, f"{url.split('/')[-1]}.csv")
        if os.path.exists(output_file) and not overwrite:
            print(f"File {output_file} already exists. Skipping URL {url}.")
            continue

        # Rotate user agent
        user_agent = random.choice(user_agents)
        browser.execute_cdp_cmd('Network.setUserAgentOverride', {"userAgent": user_agent})
        browser.get(url)
        html = browser.page_source

        pmid_pattern = re.compile(r'https://pubmed.ncbi.nlm.nih.gov/\d+')
        pmid_patter2 = re.compile(r'https://www.ncbi.nlm.nih.gov/pubmed/\d+')
        pmids = pmid_pattern.findall(html)
        pmids += pmid_patter2.findall(html)
        # also grab the Citations count  and the number in the () following it from the html
        citations_pattern = re.compile(r'Citations \(\d+\)')
        citations = citations_pattern.findall(html)
        
        if len(citations) >= 1:
            print(citations, len(pmids))
            print(pmids)
            assert len(citations) == 1
            #assert len(pmids) == int(citations[0].split('(')[1].split(')')[0])
        # if not pmids:
        #     print(f"No PMIDs found for {url}. Skipping.")
        #     continue
        if len(pmids) == 0:
            number_of_0s_count += 1
            if number_of_0s_count >= number_of_0s:
                print(f"Too many 0s in a row, resetting vpn")
                return (index - number_of_0s) + start_index
        else:
            number_of_0s_count = 0
        pmid_data = {'URL': [], 'PMID': []}
        for pmid in pmids:
            pmid_data['URL'].append(url)
            pmid_data['PMID'].append(pmid)
        # make this long when updating the daatafraame
        df = pd.DataFrame(pmid_data)
        df.to_csv(output_file, index=False)

        print(f"Saved {output_file}")

        # Delay to prevent triggering anti-bot mechanisms
        time.sleep(random.uniform(5, 10))

    browser.quit()




In [30]:
# Read URLs from CSV
file = "/Users/keithmitchell/Desktop/Repositories/NeuroMabSeq/trimmer/picklequeries/antibodiesinclinks/antibodiesinc_master-5-20-24.csv"
urls = pd.read_csv(file)['Link'].tolist()[1200:]
# Extract PMIDs and save to CSV files
output_dir = "/Users/keithmitchell/Desktop/Repositories/NeuroMabSeq/trimmer/picklequeries/antibodiesinclinks"
start_index = 0
results_index = extract_pmids(urls, output_dir, start_index, overwrite=True)
print(results_index)

<ipython-input-25-9ece17db1bde>:15: DeprecationWarning: headless property is deprecated, instead use add_argument('--headless') or add_argument('--headless=new')
  options.headless = True


WebDriverException: Message: 'chromedriver' executable needs to be in PATH. Please see https://chromedriver.chromium.org/home


# Create the master Antibodies inc dataframe

In [31]:
# create a master dataframe of the outputs from the code aboe
dfantiinc = pd.DataFrame()
for file in glob.glob("/Users/keithmitchell/Desktop/Repositories/NeuroMabSeq/trimmer/picklequeries/antibodiesinclinks/pmid_output/*.csv"):
    dfi = pd.read_csv(file)
    dfantiinc = dfantiinc.append(dfi)
    dfantiinc = dfantiinc.reset_index(drop=True)
    
for index, row in dfantiinc.iterrows():
    # look for -f with a number after it with regex
    dfantiinc.at[index, 'MABID1'] = row["URL"].split('-')[-4]
    dfantiinc.at[index, 'MABID2'] = row["URL"].split('-')[-3]
    dfantiinc.at[index, 'ANTIBODYINC1'] = row["URL"].split('-')[-2]
    dfantiinc.at[index, 'ANTIBODYINC2'] = row["URL"].split('-')[-1]
    dfantiinc.at[index, 'CONJUGATE'] = None
    # creaate a list column splitting on the last 5 objects of the url
dfantiinc['URLsplit'] = dfantiinc['URL'].str.split('-').str[-5:]
dfantiinc

<ipython-input-31-8f56fb73441f>:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfantiinc = dfantiinc.append(dfi)
<ipython-input-31-8f56fb73441f>:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfantiinc = dfantiinc.append(dfi)
<ipython-input-31-8f56fb73441f>:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfantiinc = dfantiinc.append(dfi)
<ipython-input-31-8f56fb73441f>:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfantiinc = dfantiinc.append(dfi)
<ipython-input-31-8f56fb73441f>:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfantiinc = df

,URL,PMID,MABID1,MABID2,ANTIBODYINC1,ANTIBODYINC2,CONJUGATE,URLsplit
0,https://www.antibodiesinc.com/products/anti-ne...,https://www.ncbi.nlm.nih.gov/pubmed/25429153,n126b,31,75,226,NaN,"[antibody, n126b, 31, 75, 226]"
1,https://www.antibodiesinc.com/products/anti-ne...,https://www.ncbi.nlm.nih.gov/pubmed/24229388,n126b,31,75,226,NaN,"[antibody, n126b, 31, 75, 226]"
2,https://www.antibodiesinc.com/products/anti-ne...,https://www.ncbi.nlm.nih.gov/pubmed/22249266,n126b,31,75,226,NaN,"[antibody, n126b, 31, 75, 226]"
3,https://www.antibodiesinc.com/products/anti-kv...,https://pubmed.ncbi.nlm.nih.gov/28115470,k19,36,75,012,NaN,"[antibody, k19, 36, 75, 012]"
4,https://www.antibodiesinc.com/products/anti-kv...,https://pubmed.ncbi.nlm.nih.gov/27872049,k19,36,75,012,NaN,"[antibody, k19, 36, 75, 012]"
...,...,...,...,...,...,...,...,...
5326,https://www.antibodiesinc.com/products/anti-ca...,https://pubmed.ncbi.nlm.nih.gov/24703904,n263,31,75,257,NaN,"[antibody, n263, 31, 75, 257]"
5327,https://www.antibodiesinc.com/products/anti-ca...,https://pubmed.ncbi.nlm.nih.gov/23858011,n263,31,75,257,NaN,"[antibody, n263, 31, 75, 257]"
5328,https://www.antibodiesinc.com/products/anti-ca...,https://pubmed.ncbi.nlm.nih.gov/23568894,n263,31,75,257,NaN,"[antibody, n263, 31, 75, 257]"
5329,https://www.antibodiesinc.com/products/anti-lr...,https://www.ncbi.nlm.nih.gov/pubmed/25981034,n209c,35,75,364,NaN,"[antibody, n209c, 35, 75, 364]"


In [32]:
# create column antibody inc id that joins the antibodyinc1 and antibodyinc2 columns by -
dfantiinc['ANTIBODYINC_ID'] = dfantiinc['ANTIBODYINC1'] + '-' + dfantiinc['ANTIBODYINC2']
# same for MABID but make all caps and split on /
dfantiinc['MABID'] = dfantiinc['MABID1'].str.upper() + '/' + dfantiinc['MABID2'].str.upper()
# extract the column pmid_id from the url column
dfantiinc['PMID_ID'] = dfantiinc['PMID'].str.split('/').str[-1]
# remove duplicates again
# do a groupby and add the antibody ids together in a list and the mabids 
# join the list with a comma
dfantiinc = dfantiinc.groupby('PMID_ID').agg({'ANTIBODYINC_ID': lambda x: ', '.join(set(x)), 
                                         'MABID': lambda x: ', '.join(set(x))}).reset_index()

dfantiinc

,PMID_ID,ANTIBODYINC_ID,MABID
0,16525047,"75-003, 75-017","K75/41, K55/7"
1,16566008,75-022,L6/60
2,16687515,75-038,K73/20
3,16817858,"75-026, 75-001","K87A/10, K65/35"
4,16870743,75-026,K87A/10
...,...,...,...
3298,38294129,75-029,K28/86
3299,38313055,75-188,N138/6
3300,38360747,75-029,K28/86
3301,38367666,75-010,K13/31


In [33]:
# output the antibodies INC final 
dfantiinc.to_csv("/Users/keithmitchell/Desktop/Repositories/NeuroMabSeq/trimmer/picklequeries/antibodiesinc_final-5-20-24.csv", index=False)


## Run the antibodies inc dataa

In [34]:
dfantiinc = pd.read_csv("/Users/keithmitchell/Desktop/Repositories/NeuroMabSeq/trimmer/picklequeries/antibodiesinc_final.csv")
# get all the pmids from the PMID column
pubmed_ids = dfantiinc['PMID_ID'].tolist()
for pid in pubmed_ids:
    record = fetch_details(pid)
    if record:
        continue
    else:
        print(f"For {pid}, no record was found.")
        break

In [36]:
# run the legacy data from the pubmed column
df = pd.read_excel("/Users/keithmitchell/Desktop/Repositories/NeuroMabSeq/trimmer/picklequeries/publications 041922.xlsx")
# get all the pmids from the PMID column
pubmed_ids = df['Pubmed'].tolist()
for pid in pubmed_ids:
    record = fetch_details(pid)
    if record:
        continue
    else:
        print(f"For {pid}, no record was found.")
        break


Error fetching data from PubMed for nan.
'float' object has no attribute 'split'
For nan, no record was found.


In [37]:
len(os.listdir("/Users/keithmitchell/Desktop/Repositories/NeuroMabSeq/trimmer/picklequeries/pmidjsons/"))

4540

### These seem to mostly be caused by the neuromab showing up in proximity to a target - we could search for Neuromab and the target but does not guarantee that the neuromab was the one used (though extremely likely). 
- Unfortunately scraping all papers to check proximity is not feasible as UC davis does not have access to all the papers.
- The snipped could also be used as well to see if proximal to the target. I think 99% of the time it will be relevant. 
- The one hard part is that targets and the way they are shown have weird characters and stuff so search for those with google scholar wont be super easy. 
- Will certainly merge all this previous manual work into the pipeline so the current count will add 676
- Overall my code retrieves 1185 more though (without adding just the general Neuromab search) and ensuring it is a review paper. 
- Need to remove the <sub> and </sub> from the titles and abstracts.
- Need to remove the <sup> and </sup> from the titles and abstracts.
- Need to remove the <i> and </i> from the titles and abstracts.
- Need to remove the <b> and </b> from the titles and abstracts.
- Need to remove the <u> and </u> from the titles and abstracts.
- Be more clear where the source is from for the information
- Need to fix a small subset that failed in the annotations of Pubmed info should be an easy fix
- Possible new queries:
  - Antibodies inc and target
  - Antibodies inc and target and neuromab
  - Antibodies inc and davis and target
  - Davis and target and neuromab
  - neuromab and (antibody or antibodies or mab or monoclonal) == review (if no target)
  - antibodies inc and neuromab id
     - how is antibodies inc doing this
     - search for antibodies inc and "antibody or monoclonal or mab" and "target" and each part of the catalog number. 
- Double check the merging process to include all targets and mabs for the merge with old data
- Mark which ones were exclusive to the aantibodies inc scraping
- 

In [267]:
# compare all the pmids to this file and see which we dont have
#/Users/keithmitchell/Desktop/Repositories/NeuroMabSeq/trimmer/picklequeries/publications 041922.xlsx
df = pd.read_excel("/Users/keithmitchell/Desktop/Repositories/NeuroMabSeq/trimmer/picklequeries/publications 041922.xlsx")
# filter out anything that saays review in the clone
# remove all Pubmed that are empty
df = df[df['Pubmed'].notna()]
# set the Clone to a string column and remove any that are na
df['Clone'] = df['Clone'].astype(str)
df = df[df['Clone'] != 'nan']
df = df[~df['Clone'].str.contains('review', case=False)]
print(len(set([str(i) for i in df["Pubmed"].tolist()])- set([str(i.split('.')[0]) for i in os.listdir("/Users/keithmitchell/Desktop/Repositories/NeuroMabSeq/trimmer/picklequeries/pmidjsons/")])))
set([str(i) for i in df["Pubmed"].tolist()])- set([str(i.split('.')[0]) for i in os.listdir("/Users/keithmitchell/Desktop/Repositories/NeuroMabSeq/trimmer/picklequeries/pmidjsons/")])

676


{'16511870',
 '16982420',
 '17379638',
 '17537961',
 '17656725',
 '17855588',
 '18083105',
 '18344983',
 '18458091',
 '18480286',
 '18511484',
 '18680201',
 '18716211',
 '18753128',
 '18753382',
 '18832009',
 '18995817',
 '19038221',
 '19118598',
 '19162133',
 '19167097',
 '19185209',
 '19224364',
 '19370153',
 '19558451',
 '19627441',
 '19657031',
 '19697962',
 '19717445',
 '19723784',
 '19746427',
 '19756181',
 '19795496',
 '19796685',
 '19841742',
 '19912469',
 '19945389',
 '19955355',
 '20008515',
 '20037628',
 '20075334',
 '20124353',
 '20139976',
 '20147652',
 '20178748',
 '20304926',
 '20335472',
 '20385823',
 '20451588',
 '20511282',
 '20573888',
 '20628354',
 '20805102',
 '20823221',
 '20826790',
 '20837535',
 '20850541',
 '20876567',
 '20926677',
 '20943764',
 '20946956',
 '20950357',
 '21041398',
 '21046559',
 '21115101',
 '21115966',
 '21116650',
 '21123556',
 '21145482',
 '21179406',
 '21245341',
 '21307237',
 '21307309',
 '21315255',
 '21325018',
 '21358642',
 '21362533',

In [269]:
# check which we have and the other doesnt 
print(len(set([str(i.split('.')[0]) for i in os.listdir("/Users/keithmitchell/Desktop/Repositories/NeuroMabSeq/trimmer/picklequeries/pmidjsons/")])- set([str(i) for i in df["Pubmed"].tolist()])))
set([str(i.split('.')[0]) for i in os.listdir("/Users/keithmitchell/Desktop/Repositories/NeuroMabSeq/trimmer/picklequeries/pmidjsons/")]) - set([str(i) for i in df["Pubmed"].tolist()])

1185


{'28119464',
 '34718573',
 '25691569',
 '36220889',
 '35176221',
 '37051140',
 '31142543',
 '35441593',
 '36034774',
 '31816021',
 '36097791',
 '36429074',
 '34063992',
 '31831638',
 '35701597',
 '34003113',
 '36833753',
 '28496181',
 '28158844',
 '30154505',
 '32917588',
 '26027926',
 '36358921',
 '23323617',
 '36472910',
 '37333281',
 '29398366',
 '27568555',
 '35760104',
 '35456979',
 '23404451',
 '32697988',
 '30773633',
 '37528484',
 '34705506',
 '32156701',
 '29692709',
 '26435106',
 '32801157',
 '33550628',
 '26527003',
 '33817875',
 '37848409',
 '34880118',
 '37114864',
 '34985913',
 '37704727',
 '36982541',
 '29872123',
 '33727250',
 '33854181',
 '36257688',
 '30456368',
 '34000300',
 '37425915',
 '34492310',
 '35789851',
 '36658457',
 '37048156',
 '33837049',
 '28432142',
 '37819632',
 '34030727',
 '34152608',
 '34610637',
 '35966009',
 '27134175',
 '29221132',
 '29134655',
 '33606320',
 '19409269',
 '35318302',
 '36732641',
 '36609445',
 '34500285',
 '31186137',
 '23894271',

In [270]:
# print the snippets out line by line for the google scholar search
for index, row in df_final.iterrows():
    print(row['snippet'])
    print('')

BACKGROUND: KCNQ1 (Kv7.1), together with its KCNE β subunits, plays a pivotal role 
both in the repolarization of cardiac tissue and in water and salt transport across epithelial …

… , N212/17, Neuromab, CA, USA) antibodies using a previously described protocol [36, 37]. 
Imaging analysis was performed using a Nikon A1 confocal microscope at the Northwestern …

Tetratricopeptide repeat (TPR) domains are ubiquitous structural motifs that mediate protein–protein 
interactions. For example, the TPR domains in the peroxisomal import receptor …

Hyperpolarization-activated cyclic nucleotide-gated (HCN) channels are expressed in the 
heart and central nervous system where they regulate membrane excitability. Recently, …

… N144/14; University of California Davis/National Institutes of Health … (N144/14) was 
obtained form the University of California Davis/National Institutes of Health NeuroMab Facility. …

… Lysates were incubated with CRMP-2-6xHis, following phosphorylation, and immunopre

# Final Output

## TODO merge in the old outputs we are missing as well ~600 and talk to Jim about best strategy for those

In [39]:
# read this file googescholar_mabidsearch
df_final_old= pd.read_csv("/Users/keithmitchell/Desktop/Repositories/NeuroMabSeq/trimmer/picklequeries/googescholar_mabidsearch_5-20-24.csv")
df_final_old[df_final_old['publication_info'].str.contains('Miyazaki')]

,title,title_link,publication_info,snippet,cited_by_link,cited_by_count,pdf_file,Antibody,Target
10,Chemical corrector treatment ameliorates incre...,https://www.nature.com/articles/nm.3759,"N Yokoi, Y Fukata, D Kase, T Miyazaki, M Jaegl...",Epilepsy is one of the most common and intract...,https://scholar.google.com/scholar?cites=35472...,74.0,https://www.nature.com/articles/nm.3759.pdf,N144/14,6xHis
669,Oligodendrocyte-derived LGI3 and its receptor...,https://www.cell.com/cell-reports/pdf/S2211-12...,"Y Miyazaki, T Otsuka, Y Yamagata, T Endo, M Sa...","Neurodevelopmental disorders, such as intellec...",https://scholar.google.com/scholar?cites=40492...,1.0,https://www.cell.com/cell-reports/pdf/S2211-12...,K36/15,Kv1.1 K+ channel (external)
739,Oligodendrocyte-derived LGI3 and its receptor...,https://www.cell.com/cell-reports/pdf/S2211-12...,"Y Miyazaki, T Otsuka, Y Yamagata, T Endo, M Sa...","Neurodevelopmental disorders, such as intellec...",https://scholar.google.com/scholar?cites=40492...,1.0,https://www.cell.com/cell-reports/pdf/S2211-12...,K13/31,Kv1.4 K+ channel
781,Oligodendrocyte-derived LGI3 and its receptor...,https://www.cell.com/cell-reports/pdf/S2211-12...,"Y Miyazaki, T Otsuka, Y Yamagata, T Endo, M Sa...","Neurodevelopmental disorders, such as intellec...",https://scholar.google.com/scholar?cites=40492...,1.0,https://www.cell.com/cell-reports/pdf/S2211-12...,L23/27,Kv1.3 K+ channel
811,Oligodendrocyte-derived LGI3 and its receptor...,https://www.cell.com/cell-reports/pdf/S2211-12...,"Y Miyazaki, T Otsuka, Y Yamagata, T Endo, M Sa...","Neurodevelopmental disorders, such as intellec...",https://scholar.google.com/scholar?cites=40492...,1.0,https://www.cell.com/cell-reports/pdf/S2211-12...,K14/16,Kv1.2 K+ channel
838,Oligodendrocyte-derived LGI3 and its receptor...,https://www.cell.com/cell-reports/pdf/S2211-12...,"Y Miyazaki, T Otsuka, Y Yamagata, T Endo, M Sa...","Neurodevelopmental disorders, such as intellec...",https://scholar.google.com/scholar?cites=40492...,1.0,https://www.cell.com/cell-reports/pdf/S2211-12...,N46/30,ADAM22 (cytoplasmic)
846,Proteomic analysis of native cerebellar iFGF14...,https://www.tandfonline.com/doi/abs/10.1080/19...,"MK Bosch, JM Nerbonne, RR Townsend, H Miyazaki...",Intracellular Fibroblast Growth Factor 14 (iFG...,https://scholar.google.com/scholar?cites=13497...,14.0,https://www.tandfonline.com/doi/pdf/10.1080/19...,N56/21,FGF14/FHF4
1001,Oligodendrocyte-derived LGI3 and its receptor...,https://www.cell.com/cell-reports/pdf/S2211-12...,"Y Miyazaki, T Otsuka, Y Yamagata, T Endo, M Sa...","Neurodevelopmental disorders, such as intellec...",https://scholar.google.com/scholar?cites=40492...,1.0,https://www.cell.com/cell-reports/pdf/S2211-12...,K65/35,CASPR/Neurexin IV
1036,Oligodendrocyte-derived LGI3 and its receptor...,https://www.cell.com/cell-reports/pdf/S2211-12...,"Y Miyazaki, T Otsuka, Y Yamagata, T Endo, M Sa...","Neurodevelopmental disorders, such as intellec...",https://scholar.google.com/scholar?cites=40492...,1.0,https://www.cell.com/cell-reports/pdf/S2211-12...,K67/25,CASPR2


In [40]:
df_final = pd.read_csv("/Users/keithmitchell/Desktop/Repositories/NeuroMabSeq/trimmer/picklequeries/googlescholar_mabidsearch_pubmed_5-20-24.csv")
# filter for Miyazaki
df_final[df_final['publication_info'].str.contains('Miyazaki')]

,title,title_link,publication_info,snippet,cited_by_link,cited_by_count,pdf_file,Antibody,Target,PMID,Author,Publication Date,Journal,Date,Publication
920,Oligodendrocyte-derived LGI3 and its receptor...,https://www.cell.com/cell-reports/pdf/S2211-12...,"Y Miyazaki, T Otsuka, Y Yamagata, T Endo, M Sa...","Neurodevelopmental disorders, such as intellec...",https://scholar.google.com/scholar?cites=40492...,1.0,https://www.cell.com/cell-reports/pdf/S2211-12...,K36/15,Kv1.1 K+ channel (external),38194969.0,NaN,NaN,NaN,2024-01-10,Miyazaki et al. Cell reports
969,Oligodendrocyte-derived LGI3 and its receptor...,https://www.cell.com/cell-reports/pdf/S2211-12...,"Y Miyazaki, T Otsuka, Y Yamagata, T Endo, M Sa...","Neurodevelopmental disorders, such as intellec...",https://scholar.google.com/scholar?cites=40492...,1.0,https://www.cell.com/cell-reports/pdf/S2211-12...,K13/31,Kv1.4 K+ channel,38194969.0,NaN,NaN,NaN,2024-01-10,Miyazaki et al. Cell reports
1003,Oligodendrocyte-derived LGI3 and its receptor...,https://www.cell.com/cell-reports/pdf/S2211-12...,"Y Miyazaki, T Otsuka, Y Yamagata, T Endo, M Sa...","Neurodevelopmental disorders, such as intellec...",https://scholar.google.com/scholar?cites=40492...,1.0,https://www.cell.com/cell-reports/pdf/S2211-12...,L23/27,Kv1.3 K+ channel,38194969.0,NaN,NaN,NaN,2024-01-10,Miyazaki et al. Cell reports
1026,Oligodendrocyte-derived LGI3 and its receptor...,https://www.cell.com/cell-reports/pdf/S2211-12...,"Y Miyazaki, T Otsuka, Y Yamagata, T Endo, M Sa...","Neurodevelopmental disorders, such as intellec...",https://scholar.google.com/scholar?cites=40492...,1.0,https://www.cell.com/cell-reports/pdf/S2211-12...,K14/16,Kv1.2 K+ channel,38194969.0,NaN,NaN,NaN,2024-01-10,Miyazaki et al. Cell reports
1045,Oligodendrocyte-derived LGI3 and its receptor...,https://www.cell.com/cell-reports/pdf/S2211-12...,"Y Miyazaki, T Otsuka, Y Yamagata, T Endo, M Sa...","Neurodevelopmental disorders, such as intellec...",https://scholar.google.com/scholar?cites=40492...,1.0,https://www.cell.com/cell-reports/pdf/S2211-12...,N46/30,ADAM22 (cytoplasmic),38194969.0,NaN,NaN,NaN,2024-01-10,Miyazaki et al. Cell reports
1051,Proteomic analysis of native cerebellar iFGF14...,https://www.tandfonline.com/doi/abs/10.1080/19...,"MK Bosch, JM Nerbonne, RR Townsend, H Miyazaki...",Intracellular Fibroblast Growth Factor 14 (iFG...,https://scholar.google.com/scholar?cites=13497...,14.0,https://www.tandfonline.com/doi/pdf/10.1080/19...,N56/21,FGF14/FHF4,26889602.0,NaN,NaN,NaN,2016-02-19,"Bosch et al. Channels (Austin, Tex.)"
1168,Oligodendrocyte-derived LGI3 and its receptor...,https://www.cell.com/cell-reports/pdf/S2211-12...,"Y Miyazaki, T Otsuka, Y Yamagata, T Endo, M Sa...","Neurodevelopmental disorders, such as intellec...",https://scholar.google.com/scholar?cites=40492...,1.0,https://www.cell.com/cell-reports/pdf/S2211-12...,K65/35,CASPR/Neurexin IV,38194969.0,NaN,NaN,NaN,2024-01-10,Miyazaki et al. Cell reports
1188,Oligodendrocyte-derived LGI3 and its receptor...,https://www.cell.com/cell-reports/pdf/S2211-12...,"Y Miyazaki, T Otsuka, Y Yamagata, T Endo, M Sa...","Neurodevelopmental disorders, such as intellec...",https://scholar.google.com/scholar?cites=40492...,1.0,https://www.cell.com/cell-reports/pdf/S2211-12...,K67/25,CASPR2,38194969.0,NaN,NaN,NaN,2024-01-10,Miyazaki et al. Cell reports
1894,Proteomic analysis of native cerebellar iFGF14...,https://www.tandfonline.com/doi/abs/10.1080/19...,"MK Bosch, JM Nerbonne, RR Townsend, H Miyazaki...",Intracellular Fibroblast Growth Factor 14 (iFG...,https://scholar.google.com/scholar?cites=13497...,14.0,https://www.tandfonline.com/doi/pdf/10.1080/19...,K74/71,Nav1.1 Na+ channel,26889602.0,NaN,NaN,NaN,2016-02-19,"Bosch et al. Channels (Austin, Tex.)"
1918,Proteomic analysis of native cerebellar iFGF14...,https://www.tandfonline.com/doi/abs/10.1080/19...,"MK Bosch, JM Nerbonne, RR Townsend, H Miyazaki...",Intracellular Fibroblast Growth Factor 14 (iFG...,https://scholar.google.com/scho

In [41]:
# for each row in the df_final, if df_final_old contains the title and title_link then replace target with the target from that dataframe
for index, row in df_final.iterrows():
    if len(df_final_old[(df_final_old['title'] == row['title']) & (df_final_old['title_link'] == row['title_link'])]) > 0:
        df_final.at[index, 'Target'] = df_final_old[(df_final_old['title'] == row['title']) & (df_final_old['title_link'] == row['title_link'])]['Target'].values[0]
df_final[df_final['publication_info'].str.contains('Miyazaki')]


,title,title_link,publication_info,snippet,cited_by_link,cited_by_count,pdf_file,Antibody,Target,PMID,Author,Publication Date,Journal,Date,Publication
920,Oligodendrocyte-derived LGI3 and its receptor...,https://www.cell.com/cell-reports/pdf/S2211-12...,"Y Miyazaki, T Otsuka, Y Yamagata, T Endo, M Sa...","Neurodevelopmental disorders, such as intellec...",https://scholar.google.com/scholar?cites=40492...,1.0,https://www.cell.com/cell-reports/pdf/S2211-12...,K36/15,Kv1.1 K+ channel (external),38194969.0,NaN,NaN,NaN,2024-01-10,Miyazaki et al. Cell reports
969,Oligodendrocyte-derived LGI3 and its receptor...,https://www.cell.com/cell-reports/pdf/S2211-12...,"Y Miyazaki, T Otsuka, Y Yamagata, T Endo, M Sa...","Neurodevelopmental disorders, such as intellec...",https://scholar.google.com/scholar?cites=40492...,1.0,https://www.cell.com/cell-reports/pdf/S2211-12...,K13/31,Kv1.1 K+ channel (external),38194969.0,NaN,NaN,NaN,2024-01-10,Miyazaki et al. Cell reports
1003,Oligodendrocyte-derived LGI3 and its receptor...,https://www.cell.com/cell-reports/pdf/S2211-12...,"Y Miyazaki, T Otsuka, Y Yamagata, T Endo, M Sa...","Neurodevelopmental disorders, such as intellec...",https://scholar.google.com/scholar?cites=40492...,1.0,https://www.cell.com/cell-reports/pdf/S2211-12...,L23/27,Kv1.1 K+ channel (external),38194969.0,NaN,NaN,NaN,2024-01-10,Miyazaki et al. Cell reports
1026,Oligodendrocyte-derived LGI3 and its receptor...,https://www.cell.com/cell-reports/pdf/S2211-12...,"Y Miyazaki, T Otsuka, Y Yamagata, T Endo, M Sa...","Neurodevelopmental disorders, such as intellec...",https://scholar.google.com/scholar?cites=40492...,1.0,https://www.cell.com/cell-reports/pdf/S2211-12...,K14/16,Kv1.1 K+ channel (external),38194969.0,NaN,NaN,NaN,2024-01-10,Miyazaki et al. Cell reports
1045,Oligodendrocyte-derived LGI3 and its receptor...,https://www.cell.com/cell-reports/pdf/S2211-12...,"Y Miyazaki, T Otsuka, Y Yamagata, T Endo, M Sa...","Neurodevelopmental disorders, such as intellec...",https://scholar.google.com/scholar?cites=40492...,1.0,https://www.cell.com/cell-reports/pdf/S2211-12...,N46/30,Kv1.1 K+ channel (external),38194969.0,NaN,NaN,NaN,2024-01-10,Miyazaki et al. Cell reports
1051,Proteomic analysis of native cerebellar iFGF14...,https://www.tandfonline.com/doi/abs/10.1080/19...,"MK Bosch, JM Nerbonne, RR Townsend, H Miyazaki...",Intracellular Fibroblast Growth Factor 14 (iFG...,https://scholar.google.com/scholar?cites=13497...,14.0,https://www.tandfonline.com/doi/pdf/10.1080/19...,N56/21,FGF14/FHF4,26889602.0,NaN,NaN,NaN,2016-02-19,"Bosch et al. Channels (Austin, Tex.)"
1168,Oligodendrocyte-derived LGI3 and its receptor...,https://www.cell.com/cell-reports/pdf/S2211-12...,"Y Miyazaki, T Otsuka, Y Yamagata, T Endo, M Sa...","Neurodevelopmental disorders, such as intellec...",https://scholar.google.com/scholar?cites=40492...,1.0,https://www.cell.com/cell-reports/pdf/S2211-12...,K65/35,Kv1.1 K+ channel (external),38194969.0,NaN,NaN,NaN,2024-01-10,Miyazaki et al. Cell reports
1188,Oligodendrocyte-derived LGI3 and its receptor...,https://www.cell.com/cell-reports/pdf/S2211-12...,"Y Miyazaki, T Otsuka, Y Yamagata, T Endo, M Sa...","Neurodevelopmental disorders, such as intellec...",https://scholar.google.com/scholar?cites=40492...,1.0,https://www.cell.com/cell-reports/pdf/S2211-12...,K67/25,Kv1.1 K+ channel (external),38194969.0,NaN,NaN,NaN,2024-01-10,Miyazaki et al. Cell reports
1894,Proteomic analysis of native cerebellar iFGF14...,https://www.tandfonline.com/doi/abs/10.1080/19...,"MK Bosch, JM Nerbonne, RR Townsend, H Miyazaki...",Intracellular Fibroblast Growth Factor 14 (iFG...,https://scholar.google.com/scholar?cites=13497...,14.0,https://www.tandfonline.com/doi/pdf/10.1080/19...,K74/71,FGF14/FHF4,26889602.0,NaN,NaN,NaN,2016-02-19,"Bosch et al. Channels (Austin, Tex.)"
1918,Proteomic analysis of native cerebellar iFGF14...,https://www.tandfonline.com/doi/abs/10.1080/19...,"MK Bosch, JM Nerbonne, RR Townsend, H Miyazaki...",Intracellular Fibr

In [42]:
import pandas as pd
# import BytesIO to read the data from the url
from io import BytesIO
import requests
import os

# merge df_final and dfantiinc on the pmid columns do an outer join
# df_final = pd.read_csv("/Users/keithmitchell/Desktop/Repositories/NeuroMabSeq/trimmer/picklequeries/googlescholar_mabidsearch_pubmed.csv")
dfantiinc = pd.read_csv("/Users/keithmitchell/Desktop/Repositories/NeuroMabSeq/trimmer/picklequeries/antibodiesinc_final-5-20-24.csv")
# set them both PMID and PMID_ID to int
df_final['PMID'] = df_final['PMID'].astype(int)
dfantiinc['PMID_ID'] = dfantiinc['PMID_ID'].astype(int)
df_final = df_final.merge(dfantiinc, left_on='PMID', right_on='PMID_ID', how='outer')
# create a final mab_id column that takes all the unique values from the MABID column and the Antibody column
df_final['MAB_ID'] = df_final['MABID'].fillna(df_final['Antibody'])
df_final['PMID'] = df_final['PMID'].fillna(df_final['PMID_ID'])
df_final['PMID'] = df_final['PMID'].astype(int).astype(str)

# read the cfm file like before to get the target information
data = requests.get("https://neuromab.ucdavis.edu/catalog-download.cfm").content
df = pd.read_csv(BytesIO(data))
print(df)
# for the MAB_ID column create a new column called target that is the target from the df where aall the clones match
for index, row in df_final.iterrows():
    # look for -f with a number after it with regex
    # get the list of clones in the df_final["MAB_ID"]
    clones = [i.strip() for i in row["MAB_ID"].split(',')]
    print(clones)
    print(df[df['Clone'].isin(clones)]['Target'].tolist())
    df_final.at[index, 'Target'] = df[df['Clone'].isin(clones)]['Target'].tolist()

# now add the pubmed dictionary columns to the df_final
df_cure = df_final[['MAB_ID', 'Target', 'PMID']]
# do another groupby on the PMID and join the Target column with a comma and the PMID column with a comma
# then ensure that the Target values are unique and the MAID_ID values are unique
# turn the list columns into strings with a comma by joining

# for index, row in df_cure.iterrows():
#     if type(row['Target']) == list:
#         df_cure.at[index, 'Target'] = ', '.join(set(row['Target']))
#     if type(row['MAB_ID']) == list:
#         df_cure.at[index, 'MAB_ID'] = ', '.join(set(row['MAB_ID']))

# groupby the pmid and join the target and mab_id columns with a comma but make them a set before joining
# make PMID a string
df_cure['PMID'] = df_cure['PMID'].astype(str)
# turn MAB_D and Target into lists
df_cure['MAB_ID'] = df_cure['MAB_ID'].str.split(',')
# df_cure['Target'] = df_cure['Target'].str.split(',')
# df_cure = df_cure.groupby('PMID').agg({'MAB_ID': lambda x: ', '.join(set(str(x))), "Target": lambda x: ', '.join(set(str(x)))}).reset_index()

# now turn these back into a list and make sure they are unique
# for index, row in df_cure.iterrows():
#     if type(row['Target']) == str:
#         df_cure.at[index, 'Target'] = set(row['Target'].split(', '))
#     if type(row['MAB_ID']) == str:
#         df_cure.at[index, 'MAB_ID'] = set(row['MAB_ID'].split(', '))

# add the citation column to the df_cure using the column output as a dictionary (citation title and year)
df_cure["citation"] = ""
df_cure["title"] = ""
df_cure["year"] = ""
for index, row in df_cure.iterrows():
    # look for -f with a number after it with regex
    # get the list of clones in the df_final["MAB_ID"]
    pmid = row["PMID"]
    record = fetch_details(pmid)
    # check if the record is a dictionary with the relevant keys
    if type(record) == dict and 'citation' in record and 'title' in record and 'year' in record:
        df_cure.at[index, 'citation'] = record['citation']
        df_cure.at[index, 'title'] = record['title']
        df_cure.at[index, 'year'] = record['year']
    else:
        print(f"For {pmid}, no record was found.")
        
df_cure



       TargetType                   Target AccessionNum     Clone  \
0     Epigenetics         5-formylcytidine          NaN   N358/68   
1     Epigenetics  5-hydroxymethylcytidine          NaN   N297/59   
2             Tag                    6xHis          NaN   N144/14   
3       Receptors                   ADAM11       Q9R1V4   N441/35   
4       Receptors     ADAM22 (cytoplasmic)       Q9R1V6    N46/30   
..            ...                      ...          ...       ...   
478  Rare Disease                      WRN       Q14191   N428/12   
479         Other     Zebrafish PSD Marker          NaN   N286/74   
480         Other                  Zinedin       Q9NRL3    K88/64   
481  Transporters                     ZIP3       Q99K24    N476/9   
482         Other                   ZNF423       Q2M1K9  N328B/37   

    HumanGeneName DataSheetFileName IsoType ValidationT ValidationBrIB  \
0            ----       N358_68.pdf    IgG1         NaN            NaN   
1            ----      

<ipython-input-42-def262d400c5>:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cure['PMID'] = df_cure['PMID'].astype(str)
<ipython-input-42-def262d400c5>:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cure['MAB_ID'] = df_cure['MAB_ID'].str.split(',')
<ipython-input-42-def262d400c5>:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/

Error fetching data from PubMed for 28320946.
'AuthorList'
For 28320946, no record was found.
Error fetching data from PubMed for 28320946.
'AuthorList'
For 28320946, no record was found.
Error fetching data from PubMed for 28320946.
'AuthorList'
For 28320946, no record was found.
Error fetching data from PubMed for 26186538.
'LastName'
For 26186538, no record was found.
Error fetching data from PubMed for 26186538.
'LastName'
For 26186538, no record was found.
Error fetching data from PubMed for 31816021.
'AuthorList'
For 31816021, no record was found.
Error fetching data from PubMed for 31816021.
'AuthorList'
For 31816021, no record was found.
Error fetching data from PubMed for 31816021.
'AuthorList'
For 31816021, no record was found.


,MAB_ID,Target,PMID,citation,title,year
0,[N37A/10],[Kv7.1/KCNQ1 K+ channel],22024150,Krzystanek K et al. 2012 Heart Rhythm 9 440-8.,Deubiquitylating enzyme USP2 counteracts Nedd4...,2012
1,[N70/28],[HCN1],29427087,Frigerio F et al. 2018 Mol Neurobiol 55 7500-7...,Neuroinflammation Alters Integrative Propertie...,2018
2,[N212/17],[TRIP8b (constant)],28887304,Lyman KA et al. 2017 J Biol Chem 292 17718-17730.,Allostery between two binding sites in the ion...,2017
3,[N212/17],[TRIP8b (constant)],28127990,Lyman KA et al. 2017 Expert Opin Ther Targets ...,Animal models suggest the TRIP8b-HCN interacti...,2017
4,[N144/14],[6xHis],22184198,Emond MR et al. 2011 J Cell Biol 195 1115-21.,A complex of Protocadherin-19 and N-cadherin m...,2011
...,...,...,...,...,...,...
5557,[N29/29],[VGlut2],38219919,Go J et al. 2024 Life Sci 339 122413.,Agathobaculum butyriciproducens improves agein...,2024
5558,[K28/86],[Pan-MAGUK],38294129,Lee DW et al. 2024 Muscle Nerve 69 498-503.,Protective effect of alpha-lipoic acid and epa...,2024
5559,[K28/86],[Pan-MAGUK],38360747,Delhaye M et al. 2024 J Neurosci 44.,Adaptation of Magnified Analysis of the Proteo...,2024
5560,[K13/31],[Kv1.4 K+ channel],38367666,Kabakov AY et al. 2024 J Biol Chem 300 105759.,E3 ubiquitin ligase rififylin has yin and yang...,2024


In [43]:
# filter df_cure for the same PMID
df_cure[df_cure['PMID'] == "38194969"]

,MAB_ID,Target,PMID,citation,title,year
1845,"[K67/25, K36/15, K13/31, L23/27]","[CASPR2, Kv1.1 K+ channel (external), Kv1.3 K+...",38194969,Miyazaki Y et al. 2024 Cell Rep 43 113634.,Oligodendrocyte-derived LGI3 and its receptor ...,2024
1846,"[K67/25, K36/15, K13/31, L23/27]","[CASPR2, Kv1.1 K+ channel (external), Kv1.3 K+...",38194969,Miyazaki Y et al. 2024 Cell Rep 43 113634.,Oligodendrocyte-derived LGI3 and its receptor ...,2024
1847,"[K67/25, K36/15, K13/31, L23/27]","[CASPR2, Kv1.1 K+ channel (external), Kv1.3 K+...",38194969,Miyazaki Y et al. 2024 Cell Rep 43 113634.,Oligodendrocyte-derived LGI3 and its receptor ...,2024
1848,"[K67/25, K36/15, K13/31, L23/27]","[CASPR2, Kv1.1 K+ channel (external), Kv1.3 K+...",38194969,Miyazaki Y et al. 2024 Cell Rep 43 113634.,Oligodendrocyte-derived LGI3 and its receptor ...,2024
1849,"[K67/25, K36/15, K13/31, L23/27]","[CASPR2, Kv1.1 K+ channel (external), Kv1.3 K+...",38194969,Miyazaki Y et al. 2024 Cell Rep 43 113634.,Oligodendrocyte-derived LGI3 and its receptor ...,2024
1850,"[K67/25, K36/15, K13/31, L23/27]","[CASPR2, Kv1.1 K+ channel (external), Kv1.3 K+...",38194969,Miyazaki Y et al. 2024 Cell Rep 43 113634.,Oligodendrocyte-derived LGI3 and its receptor ...,2024
1851,"[K67/25, K36/15, K13/31, L23/27]","[CASPR2, Kv1.1 K+ channel (external), Kv1.3 K+...",38194969,Miyazaki Y et al. 2024 Cell Rep 43 113634.,Oligodendrocyte-derived LGI3 and its receptor ...,2024
1852,"[K67/25, K36/15, K13/31, L23/27]","[CASPR2, Kv1.1 K+ channel (external), Kv1.3 K+...",38194969,Miyazaki Y et al. 2024 Cell Rep 43 113634.,Oligodendrocyte-derived LGI3 and its receptor ...,2024


In [44]:
# lets add the legacy info so iterate through the df and see if the pubmed is not in the df_cure, if not append it
df = pd.read_excel("/Users/keithmitchell/Desktop/Repositories/NeuroMabSeq/trimmer/picklequeries/publications 041922.xlsx")
# filter out anything that saays review in the clone
# remove all Pubmed that are empty
df = df[df['Pubmed'].notna()]
# set the Clone to a string column and remove any that are na
df['Clone'] = df['Clone'].astype(str)
df = df[df['Clone'] != 'nan']
df_cure["source"] = ""

for index, row in df.iterrows():
    # look for -f with a number after it with regex
    # get the list of clones in the df_final["MAB_ID"]
    pmid = str(row["Pubmed"]).split('.')[0]
    if pmid not in df_cure['PMID'].tolist():
        record = fetch_details(pmid)
        # check if the record is a dictionary with the relevant keys
        if type(record) == dict and 'citation' in record and 'title' in record and 'year' in record:
            
            # use the clone column for MAB_ID and the target column for Target just make them a set after splitting on the comma
            # check if both are not na first
            if type(row['Clone']) == str and type(row['Target']) == str:
                df_cure = df_cure.append({'PMID': pmid, 'citation': record['citation'], 'title': record['title'], 'year': record['year'], 'source':'legacy', 'MAB_ID': set(row['Clone'].split(', ')), 'Target': set(row['Target'].split(', '))}, ignore_index=True)
            else:
                df_cure = df_cure.append({'PMID': pmid, 'citation': record['citation'], 'title': record['title'], 'year': record['year'], 'source':'legacy', 'MAB_ID': row['Clone'], 'Target': row['Target']}, ignore_index=True)

        else:
            print(f"For {pmid}, no record was found.")


# now also iterate through the df_cure is not in the df, if so update the source column to google scholar unique
for index, row in df_cure.iterrows():
    # look for -f with a number after it with regex
    # get the list of clones in the df_final["MAB_ID"]
    pmid = str(row["PMID"])
    if pmid not in df['Pubmed'].astype(str).tolist():
        df_cure.at[index, 'source'] = 'google scholar unique'



<ipython-input-44-b8b1931b25da>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cure["source"] = ""
<ipython-input-44-b8b1931b25da>:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_cure = df_cure.append({'PMID': pmid, 'citation': record['citation'], 'title': record['title'], 'year': record['year'], 'source':'legacy', 'MAB_ID': set(row['Clone'].split(', ')), 'Target': set(row['Target'].split(', '))}, ignore_index=True)
<ipython-input-44-b8b1931b25da>:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_cure = df_cure.append({'PMID': pmid, 'citation': record['citati

Error fetching data from PubMed for 24077800.
list index out of range
For 24077800, no record was found.


<ipython-input-44-b8b1931b25da>:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_cure = df_cure.append({'PMID': pmid, 'citation': record['citation'], 'title': record['title'], 'year': record['year'], 'source':'legacy', 'MAB_ID': set(row['Clone'].split(', ')), 'Target': set(row['Target'].split(', '))}, ignore_index=True)
<ipython-input-44-b8b1931b25da>:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_cure = df_cure.append({'PMID': pmid, 'citation': record['citation'], 'title': record['title'], 'year': record['year'], 'source':'legacy', 'MAB_ID': set(row['Clone'].split(', ')), 'Target': set(row['Target'].split(', '))}, ignore_index=True)
<ipython-input-44-b8b1931b25da>:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
 

In [45]:
df_cure

,MAB_ID,Target,PMID,citation,title,year,source
0,[N37A/10],[Kv7.1/KCNQ1 K+ channel],22024150,Krzystanek K et al. 2012 Heart Rhythm 9 440-8.,Deubiquitylating enzyme USP2 counteracts Nedd4...,2012,
1,[N70/28],[HCN1],29427087,Frigerio F et al. 2018 Mol Neurobiol 55 7500-7...,Neuroinflammation Alters Integrative Propertie...,2018,
2,[N212/17],[TRIP8b (constant)],28887304,Lyman KA et al. 2017 J Biol Chem 292 17718-17730.,Allostery between two binding sites in the ion...,2017,google scholar unique
3,[N212/17],[TRIP8b (constant)],28127990,Lyman KA et al. 2017 Expert Opin Ther Targets ...,Animal models suggest the TRIP8b-HCN interacti...,2017,google scholar unique
4,[N144/14],[6xHis],22184198,Emond MR et al. 2011 J Cell Biol 195 1115-21.,A complex of Protocadherin-19 and N-cadherin m...,2011,
...,...,...,...,...,...,...,...
6254,{N106/65},{Ankyrin-G},33536249,Ogawa Y et al. 2021 J Cell Sci 134.,Endogenously expressed Ranbp2 is not at the ax...,2021,legacy
6255,{L6/60},{Slo1/BKAlpha},33556372,Bukiya AN et al. 2021 J Biol Chem 296 100381.,Cholesterol activates BK channels by increasin...,2021,legacy
6256,{N212B},{TRIP8b},33547341,Rátkai A et al. 2021 Sci Rep 11 3236.,Homeostatic plasticity and burst activity are ...,2021,legacy
6257,{K28/43},{PSD-95},33663561,Perkins EM et al. 2021 Mol Neurodegener 16 13.,Altered network properties in C9ORF72 repeat e...,2021,legacy


In [46]:
# summarize the source column with a figure 
import plotly.express as px
fig = px.histogram(df_cure, x="source")
fig.show()

In [155]:
# summarize the source column with a figure 
import plotly.express as px
fig = px.histogram(df_cure, x="source")
fig.show()

In [47]:
# plot based on year
import plotly.express as px
fig = px.histogram(df_cure, x="year")
fig.show()

TypeError: __new__() missing 2 required positional arguments: 'tag' and 'attributes'

In [156]:
# plot based on year
import plotly.express as px
fig = px.histogram(df_cure, x="year")
fig.show()

In [48]:
# filter year that is ''
df_cure[df_cure['year'] == '']

,MAB_ID,Target,PMID,citation,title,year,source
1560,[N95/35],"[GABA(A)R, Alpha1]",28320946,,,,google scholar unique
1561,[N87/25],"[GABA(A)R, Beta3]",28320946,,,,google scholar unique
1562,"[N95/35, N87/25]","[GABA(A)R, Alpha1, GABA(A)R, Beta3]",28320946,,,,google scholar unique
1909,[K72/8],[Co-Rest/RCOR1],26186538,,,,google scholar unique
1910,[K72/8],[Co-Rest/RCOR1],26186538,,,,google scholar unique
2246,[L21/32],[GluA2/GluR2 glutamate receptor],31816021,,,,google scholar unique
2247,[N59/36],[GluN2B/NR2B glutamate receptor],31816021,,,,google scholar unique
2248,"[N59/36, L21/32]","[GluA2/GluR2 glutamate receptor, GluN2B/NR2B g...",31816021,,,,google scholar unique


In [49]:
# output df_cure
# convert to int
# fill empty year with 0
df_cure['year'] = df_cure['year'].replace('', 0)
df_cure['year'] = df_cure['year'].astype(int)
df_cure = df_cure[df_cure['year'] >= 2000]

df_cure.to_csv("/Users/keithmitchell/Desktop/Repositories/NeuroMabSeq/trimmer/picklequeries/df_cure-5-20-24.csv", index=False)

In [50]:
# expand out the list columns into multiple rows (do MAB_ID)
df_cure = pd.read_csv("/Users/keithmitchell/Desktop/Repositories/NeuroMabSeq/trimmer/picklequeries/df_cure-5-20-24.csv")
# turn the string columns into lists after getting rid of the {} for MAB_ID 
# some are split on or as well fix that by replacing or with a comma space
# df_cure['MAB_ID'] = df_cure['MAB_ID'].str.replace('{', '').str.replace('}', '').str.replace("'", '').str.replace(' or ', ', ').str.split(', ')
# also reaplce the [ in the 
# replace the html stuff in the title that <>
df_cure['title'] = df_cure['title'].str.replace('<i>', '').str.replace('</i>', '').str.replace('<b>', '').str.replace('</b>', '')
df_cure['title'] = df_cure['title'].str.replace('<sub>', '').str.replace('</sub>', '').str.replace('<sup>', '').str.replace('</sup>', '')
df_cure['title'] = df_cure['title'].str.replace('<u>', '').str.replace('</u>', '')
# output a df_cure_cleaaned
# turn year into an int
#df_cure['year'] = df_cure['year'].astype(int)
df_cure.to_csv("/Users/keithmitchell/Desktop/Repositories/NeuroMabSeq/trimmer/picklequeries/df_cure_cleaned-5-20-24.csv", index=False)
df_cure = df_cure.explode('MAB_ID')
df_cure

,MAB_ID,Target,PMID,citation,title,year,source
0,['N37A/10'],['Kv7.1/KCNQ1 K+ channel'],22024150,Krzystanek K et al. 2012 Heart Rhythm 9 440-8.,Deubiquitylating enzyme USP2 counteracts Nedd4...,2012,NaN
1,['N70/28'],['HCN1'],29427087,Frigerio F et al. 2018 Mol Neurobiol 55 7500-7...,Neuroinflammation Alters Integrative Propertie...,2018,NaN
2,['N212/17'],['TRIP8b (constant)'],28887304,Lyman KA et al. 2017 J Biol Chem 292 17718-17730.,Allostery between two binding sites in the ion...,2017,google scholar unique
3,['N212/17'],['TRIP8b (constant)'],28127990,Lyman KA et al. 2017 Expert Opin Ther Targets ...,Animal models suggest the TRIP8b-HCN interacti...,2017,google scholar unique
4,['N144/14'],['6xHis'],22184198,Emond MR et al. 2011 J Cell Biol 195 1115-21.,A complex of Protocadherin-19 and N-cadherin m...,2011,NaN
...,...,...,...,...,...,...,...
6243,{'N106/65'},{'Ankyrin-G'},33536249,Ogawa Y et al. 2021 J Cell Sci 134.,Endogenously expressed Ranbp2 is not at the ax...,2021,legacy
6244,{'L6/60'},{'Slo1/BKAlpha'},33556372,Bukiya AN et al. 2021 J Biol Chem 296 100381.,Cholesterol activates BK channels by increasin...,2021,legacy
6245,{'N212B'},{'TRIP8b'},33547341,Rátkai A et al. 2021 Sci Rep 11 3236.,Homeostatic plasticity and burst activity are ...,2021,legacy
6246,{'K28/43'},{'PSD-95'},33663561,Perkins EM et al. 2021 Mol Neurodegener 16 13.,Altered network properties in C9ORF72 repeat e...,2021,legacy


In [51]:
# create a new column that is the citation_final
# make it blank for now
df_cure['citation_final'] = ''
# iterate through each row get the pubmed id and grab the corresponding json and get a citation in the following format Anderson, C.N.G., et al. 2006. J Physiol. 575:367-72.
for index, row in df_cure.iterrows():
    # look for -f with a number after it with regex
    # get the list of clones in the df_final["MAB_ID"]
    pmid = row["PMID"]
    record = fetch_details(pmid)
    # set the citation_final column to the citation value
    if type(record) == dict and 'citation' in record and 'title' in record and 'year' in record:
        df_cure.at[index, 'citation_final'] = record['citation']
    else:
        print(f"For {pmid}, no record was found.")
    # # get the first author et al. 
    # # check if the record is a dictionary with the relevant keys
    # if type(record) == dict and 'citation' in record and 'title' in record and 'year' in record:
    #     df_cure.at[index, 'citation_final'] = record['citation']
    # else:
    #     print(f"For {pmid}, no record was found.")


In [52]:
df_cure

,MAB_ID,Target,PMID,citation,title,year,source,citation_final
0,['N37A/10'],['Kv7.1/KCNQ1 K+ channel'],22024150,Krzystanek K et al. 2012 Heart Rhythm 9 440-8.,Deubiquitylating enzyme USP2 counteracts Nedd4...,2012,NaN,Krzystanek K et al. 2012 Heart Rhythm 9 440-8.
1,['N70/28'],['HCN1'],29427087,Frigerio F et al. 2018 Mol Neurobiol 55 7500-7...,Neuroinflammation Alters Integrative Propertie...,2018,NaN,Frigerio F et al. 2018 Mol Neurobiol 55 7500-7...
2,['N212/17'],['TRIP8b (constant)'],28887304,Lyman KA et al. 2017 J Biol Chem 292 17718-17730.,Allostery between two binding sites in the ion...,2017,google scholar unique,Lyman KA et al. 2017 J Biol Chem 292 17718-17730.
3,['N212/17'],['TRIP8b (constant)'],28127990,Lyman KA et al. 2017 Expert Opin Ther Targets ...,Animal models suggest the TRIP8b-HCN interacti...,2017,google scholar unique,Lyman KA et al. 2017 Expert Opin Ther Targets ...
4,['N144/14'],['6xHis'],22184198,Emond MR et al. 2011 J Cell Biol 195 1115-21.,A complex of Protocadherin-19 and N-cadherin m...,2011,NaN,Emond MR et al. 2011 J Cell Biol 195 1115-21.
...,...,...,...,...,...,...,...,...
6243,{'N106/65'},{'Ankyrin-G'},33536249,Ogawa Y et al. 2021 J Cell Sci 134.,Endogenously expressed Ranbp2 is not at the ax...,2021,legacy,Ogawa Y et al. 2021 J Cell Sci 134.
6244,{'L6/60'},{'Slo1/BKAlpha'},33556372,Bukiya AN et al. 2021 J Biol Chem 296 100381.,Cholesterol activates BK channels by increasin...,2021,legacy,Bukiya AN et al. 2021 J Biol Chem 296 100381.
6245,{'N212B'},{'TRIP8b'},33547341,Rátkai A et al. 2021 Sci Rep 11 3236.,Homeostatic plasticity and burst activity are ...,2021,legacy,Rátkai A et al. 2021 Sci Rep 11 3236.
6246,{'K28/43'},{'PSD-95'},33663561,Perkins EM et al. 2021 Mol Neurodegener 16 13.,Altered network properties in C9ORF72 repeat e...,2021,legacy,Perkins EM et al. 2021 Mol Neurodegener 16 13.


In [53]:
# filter ADAM22 contained in Target
df_cure[df_cure['PMID']==38194969]
df_curec = df_cure

In [54]:
df_curec.dtypes

MAB_ID            object
Target            object
PMID               int64
citation          object
title             object
year               int64
source            object
citation_final    object
dtype: object

In [187]:
# conver both MAB_ID and Target to strings and remove the {} and [] and split on the comma to make them a list
# df_curec['MAB_ID'] = df_cure['MAB_ID'].str.replace('{', '').str.replace('}', '').str.replace("'", '').str.replace("]", '').str.replace("[", '').str.replace(' or ', ', ').str.split(',')
# df_curec['Target'] = df_cure['Target'].str.replace('{', '').str.replace('}', '').str.replace("'", '').str.replace("]", '').str.replace("[", '').str.replace(' or ', ', ').str.split(',')


<ipython-input-187-efce0a3337cd>:2: FutureWarning:

The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.

<ipython-input-187-efce0a3337cd>:3: FutureWarning:

The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.



In [195]:
df_curec[df_curec['PMID']==38194969]
# get the column data types
df_curec.dtypes

MAB_ID            object
Target            object
PMID               int64
citation          object
title             object
year               int64
source            object
citation_final    object
dtype: object

In [55]:

# make the MAB_ID and Target columns sets 

df_curegb = df_curec.groupby('PMID').agg({'MAB_ID': lambda x: set(x), 'Target': lambda x: set(x), 'citation_final': 'first'}).reset_index()
df_curegb['MAB_ID'] = df_curegb['MAB_ID'].astype(str).str.replace('{', '').str.replace('}', '').str.replace("'", '').str.replace("]", '').str.replace("[", '')
df_curegb['Target'] = df_curegb['Target'].astype(str).str.replace('{', '').str.replace('}', '').str.replace("'", '').str.replace("]", '').str.replace("[", '')
# remove "" from Target 
df_curegb['Target'] = df_curegb['Target'].str.replace('"', '')
df_curegb

<ipython-input-55-8a05630c10f7>:4: FutureWarning:

The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.

<ipython-input-55-8a05630c10f7>:5: FutureWarning:

The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.



,PMID,MAB_ID,Target,citation_final
0,16511870,"""K28/43""",PSD-95,Mo Z et al. 2006 J Neurosci Res 83 1323-30.
1,16525047,"""K75/41, K55/7""","KChIP1 K+ channel, Kv4.3 K+ channel",Kollo M et al. 2006 J Neurosci 26 2684-91.
2,16566008,"""L6/60""",Slo1/BKAlpha maxi-K+ channel,Misonou H et al. 2006 J Comp Neurol 496 289-302.
3,16687515,"""K73/20""",Contactin/F3,Ogawa Y et al. 2006 J Neurosci 26 5230-9.
4,16793899,"""review article""",review article,Anderson CN et al. 2006 J Physiol 575 367-72.
...,...,...,...,...
4586,38736483,"""N59/36"", ""K28/43""","GluN2B/NR2B glutamate receptor, PSD-95",Matsumoto Y et al. 2024 Front Mol Neurosci 17 ...
4587,38746081,"""N106/36"", ""N56/21"", ""K87A/10""","Ankyrin-G (staining), Nav1.6 Na+ channel, FGF1...",Ransdell JL et al. 2024 bioRxiv.
4588,38767050,"""N295B/66""",Arl13b,Kumari A et al. 2024 Cytoskeleton (Hoboken).
4589,38767486,"""K28/74""",PSD-95,Sun SY et al. 2025 Neural Regen Res 20 209-223.


In [56]:
# filter for the PMID again
df_curegb[df_curegb['PMID'] == 38194969]

,PMID,MAB_ID,Target,citation_final
4530,38194969,"""K67/25, K36/15, K13/31, L23/27""","CASPR2, Kv1.1 K+ channel (external), Kv1.3 K+ ...",Miyazaki Y et al. 2024 Cell Rep 43 113634.


In [57]:
# output this to a csv file
df_curegb.to_csv("/Users/keithmitchell/Desktop/Repositories/NeuroMabSeq/trimmer/picklequeries/df_cure_cleaned_grouped-5-20-24.csv", index=False)

In [58]:
# remove anything before the year 2000
df_cure = df_cure[df_cure['year'] >= 2000]
# make the same plot
fig = px.histogram(df_cure, x="year", color="MAB_ID")
fig.show()




In [59]:
# remove anything before the year 2000
df_cure = df_cure[df_cure['year'] >= 2000]
# make the same plot
fig = px.histogram(df_cure, x="year", color="source")
# add a title that shows the neuromab generaal has not been done yet
fig.update_layout(title_text='NeuroMab General has not been done yet')
fig.show()